In [1]:
#IndicatorCode	IndicatorName
#Ц01	Доля магистров и аспирантов в общем контингенте 
#Ц02	Средний балл ЕГЭ
#Ц03	Доля сторонних магистров и аспирантов
#Ц04	Доля целевиков
#Ц05	Объем ПОУ на ставку НПР
#Ц06	Доля остепененных ППС
#Ц07	Трудоустройство
#Ц08	Количество публикаций в WoS и Scopus и журналов из перечня ВАК на ставку НПР
#Ц09	Объем НИОКР на ставку НПР
#Ц10	Защиты аспирантов в срок
#Ц11	Доля иностранных студентов
#Ц12	Количество иностранных преподавателей
#Ц13	Академическая мобильность студентов и аспирантов
#Ц16	Заявочная активность
#Ц17	Подготовленные к публикации тематические материалы  научно-просветительского характера для неспециализированных СМИ
#Ц18	Подготовка энциклопедии/справочника
#Ц20	Руководство работами студентов и аспирантов – победителей олимпиад (конкурсов, выставок)
#Ц21	Участие в научно-технических и творческих выставках (учитывается только участие с экспонатами)
##Ц22	Членство в научно-технических или учебно-методических и редакционных советах, редколлегиях
#Ц23	Членство в программных и организационных комитетах конференций и олимпиад
#Ц24	Инновации
#Ц25	Доля ППС, СЗП которых по итогам календарного года составляет 200% и более от региональной
#Ц26	Доля общей численности НПР до 39 лет к общей численности НПР (по головам)
#Ц27	РИД

In [2]:
import pyodbc 
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import math
    

In [5]:
server = 'KPI-MONITOR'
database = 'MEI2'
username = 'efqm'
password = 'mpeiR@dar'
current_date = datetime.datetime.now().date()

In [7]:
def sorted_column_to_string(df, column_name):
    """
    Функция сортирует значения указанного столбца DataFrame по возрастанию
    и возвращает их в виде строки, разделенной точкой с запятой и пробелом.
    
    :param df: DataFrame, содержащий данные
    :param column_name: Название столбца для сортировки и вывода
    :return: Строка отсортированных значений столбца
    """
    sorted_values = df[column_name].sort_values(ascending=True)  # Сортируем по возрастанию
    return "; ".join(map(str, sorted_values.tolist()))  # Преобразуем в строку


In [9]:
def analyze_distribution(df, column_name):
    """
    Эта функция рассчитывает статистические показатели для указанного столбца в DataFrame.
    Она также определяет характер распределения и выявляет выбросы на основе распределения.

    Параметры:
        df (pd.DataFrame): DataFrame, содержащий данные.
        column_name (str): Название столбца для анализа.

    Возвращает:
        None: Печатает результаты построчно, каждая статистика выводится отдельно.
    """
    data = df[column_name].dropna()  # Убираем NaN значения для корректного расчета
    
    # Рассчитываем базовые статистические показатели
    minimum = data.min()
    maximum = data.max()
    median = data.median()
    
    # Квартильные значения
    q1 = data.quantile(0.25)
    q3 = data.quantile(0.75)
    iqr = q3 - q1  # Межквартильный размах
    
    # Определение выбросов (значения выше 105% считаются выбросами)
    outliers = data[data > 105]
    
    # Определение типа скошенности
    skewness_type = "правостороннее скошенное распределение" if data.mean() < data.max() else "приблизительно симметричное"
    
    # Печать результатов построчно
    print("Минимум:", minimum)
    print("Максимум:", maximum)
    print("Медиана (Q2):", median)
    print("Q1 (25-й перцентиль):", q1)
    print("Q3 (75-й перцентиль):", q3)
    print("Межквартильный размах (IQR):", iqr)
    print("Тип скошенности:", skewness_type)
    print("Выбросы:", ', '.join(map(str, outliers.tolist())) if not outliers.empty else "Нет выбросов")



In [11]:
def build_scaleOld(percentages, num_grades=10, min_score=50, max_score=80, scale_type='linear'):
    """
    Функция для построения шкалы присвоения баллов на основе распределения данных.

    :param percentages: список процентных значений выполнения плана
    :param num_grades: количество градаций шкалы (по умолчанию 10)
    :param min_score: минимальный балл (по умолчанию 50)
    :param max_score: максимальный балл (по умолчанию 80)
    :param scale_type: тип масштабирования ('linear' или 'log')
    :return: DataFrame с диапазонами процентов и соответствующими баллами
    """
    # Преобразуем список в numpy array
    data = np.array(percentages)

    # Обрабатываем данные в зависимости от выбранного масштабирования
    if scale_type == 'log':
        # Добавляем небольшое число, чтобы избежать логарифма от нуля
        data = np.log(data + 1)
    elif scale_type != 'linear':
        raise ValueError("scale_type должен быть 'linear' или 'log'")

    # Сортируем данные
    data_sorted = np.sort(data)

    # Вычисляем перцентили для разделения на градации
    percentiles = np.linspace(0, 100, num_grades + 1)
    bins = np.percentile(data_sorted, percentiles)

    # Создаем диапазоны процентов
    ranges = []
    for i in range(len(bins) - 1):
        lower = bins[i]
        upper = bins[i + 1]

        # Преобразуем границы обратно, если использовали логарифмическое масштабирование
        if scale_type == 'log':
            lower_display = np.exp(lower) - 1
            upper_display = np.exp(upper) - 1
        else:
            lower_display = lower
            upper_display = upper

        if i == 0:
            range_str = f"≤ {upper_display:.2f}%"
        elif i == len(bins) - 2:
            range_str = f"> {lower_display:.2f}%"
        else:
            range_str = f"{lower_display:.2f}% – {upper_display:.2f}%"
        ranges.append((lower_display, upper_display, range_str))

    # Присваиваем баллы каждому диапазону
    scores = np.linspace(min_score, max_score, num_grades)

    # Создаем DataFrame для шкалы
    scale_df = pd.DataFrame({
        'Процент выполнения плана (%)': [r[2] for r in ranges],
        'Баллы за выполнение': scores.astype(int)
    })

    return scale_df



In [13]:
def build_scale_quartiles(percentages, num_sections=4, min_score=50, max_score=80):
    """
    Функция для построения шкалы присвоения баллов на основе квартильного или перцентильного разделения данных.

    :param percentages: список процентных значений выполнения плана
    :param num_sections: количество секций для разделения (по умолчанию 4 для квартилей)
    :param min_score: минимальный балл (по умолчанию 50)
    :param max_score: максимальный балл (по умолчанию 80)
    :return: DataFrame с диапазонами процентов и соответствующими баллами
    """
    df_unique = list(set(percentages.tolist()))
  

   
    # Преобразуем список в numpy array, отфильтровав только числовые значения
    data = np.array([float(value) for value in df_unique if isinstance(value, (int, float, str)) and str(value).replace('.', '', 1).isdigit()])
    data_sorted = np.sort(data)

    # Вычисляем перцентили для разделения на секции
    percentiles = np.linspace(0, 100, num_sections + 1)
    percentile_values = np.percentile(data_sorted, percentiles)

    # Создаем диапазоны на основе перцентилей
    ranges = []
    for i in range(len(percentile_values) - 1):
        lower = percentile_values[i]
        upper = percentile_values[i + 1]
        if i == 0:
            range_str = f"≤ {upper:.2f}%"
        elif i == len(percentile_values) - 2:
            range_str = f"> {lower:.2f}%"
        else:
            range_str = f"{lower:.2f}% – {upper:.2f}%"
        ranges.append((lower, upper, range_str))

    # Присваиваем баллы каждому диапазону
    num_ranges = len(ranges)
    scores = np.linspace(min_score, max_score, num_ranges)

    # Создаем DataFrame для шкалы
    scale_df = pd.DataFrame({
        'Процент выполнения плана (%)': [r[2] for r in ranges],
        'Баллы за выполнение': scores.astype(int)
    })

    return scale_df

In [15]:
def build_scale(percentages, num_grades=10, min_score=10, max_score=100, scale_type='linear'):
    """
    Функция для построения шкалы присвоения баллов на основе распределения данных и заданных критериев,
    с удалением одинаковых интервалов.

    :param percentages: список процентных значений выполнения плана
    :param num_grades: количество градаций шкалы (по умолчанию 10)
    :param min_score: минимальный балл (по умолчанию 10)
    :param max_score: максимальный балл (по умолчанию 100)
    :param scale_type: тип масштабирования ('linear' или 'log')
    :return: DataFrame с диапазонами процентов и соответствующими баллами
    """
    df_unique = list(set(percentages.tolist()))
  

    # Преобразуем список в numpy array
    data = np.array(df_unique)

    # Проверяем максимальное значение
    max_percent = max(data)

    # Обрабатываем данные в зависимости от выбранного масштабирования
    if scale_type == 'log':
        data = np.log(data + 1)  # Логарифмическое масштабирование
    elif scale_type != 'linear':
        raise ValueError("scale_type должен быть 'linear' или 'log'")

    # Сортируем данные
    data_sorted = np.sort(data)

    # Разделяем данные на перцентили
    percentiles = np.linspace(0, 100, num_grades + 1)  # Делим на num_grades диапазонов
    bins = np.percentile(data_sorted, percentiles)

    # Создаем диапазоны процентов
    ranges = []
    scores = []
    for i in range(len(bins) - 1):
        lower = bins[i]
        upper = bins[i + 1]

        # Преобразуем границы обратно, если использовали логарифмическое масштабирование
        if scale_type == 'log':
            lower_display = np.exp(lower) - 1
            upper_display = np.exp(upper) - 1
        else:
            lower_display = lower
            upper_display = upper

        if i == 0:
            range_str = f"≤ {upper_display:.2f}%"
        elif i == len(bins) - 2:
            range_str = f"> {lower_display:.2f}%"
        else:
            range_str = f"{lower_display:.2f}% – {upper_display:.2f}%"

        # Пропускаем повторяющиеся диапазоны
        #if ranges and ranges[-1][1] == upper_display:
        #    continue

        ranges.append((lower_display, upper_display, range_str))

        # Присваиваем баллы
        if upper <= 100:  # Невыполнение плана (до 100%)
            score = min_score + (50 - min_score) * (upper / 100)
        else:  # От 100% равномерное распределение
            relative_position = (upper - 100) / (max_percent - 100)
            score = 50 + (max_score - 50) * relative_position

        scores.append(round(score))  # Округляем до целого

    # Создаем DataFrame для шкалы
    scale_df = pd.DataFrame({
        'Процент выполнения плана (%)': [r[2] for r in ranges],
        'Баллы за выполнение': scores
    })

    return scale_df

In [17]:
def format_scale0(scale_df, scale_name):
    """
    Генерирует отформатированный список на основе DataFrame с указанием имени переменной.

    Args:
        scale_df (pd.DataFrame): DataFrame с диапазонами и баллами.
        scale_name (str): Имя переменной для итогового списка.

    Returns:
        str: Отформатированный список в виде строки.
    """
    formatted_scale = []
    ranges = scale_df['Процент выполнения плана (%)']
    scores = scale_df['Баллы за выполнение']
    
    # Смещаем баллы на одну позицию
    shifted_scores = list(scores[1:]) + [scores.iloc[-1]]  # Последнее значение дублируется для крайних случаев
    
    for i in range(len(ranges)):
        range_str = ranges[i]
        score = shifted_scores[i]  # Используем сдвинутый список
        
        # Извлекаем диапазоны из строк формата "x% – y%" или "> x%"
        if " – " in range_str:
            lower, upper = map(lambda x: float(x.strip('%')), range_str.split(" – "))
            if lower != upper:  # Пропускаем одинаковые нижнюю и верхнюю границы
                formatted_scale.append((upper, score))
        elif "> " in range_str:
            lower = float(range_str.split("> ")[1].strip('%'))
            formatted_scale.append((lower, score))  # Больше, чем нижняя граница
        elif "≤ " in range_str:
            upper = float(range_str.split("≤ ")[1].strip('%'))
            formatted_scale.append((upper, score))  # Меньше или равно верхней границе

    # Удаляем дубликаты значений с одинаковой границей и сортируем
    filtered_scale = []
    for i, (threshold, score) in enumerate(formatted_scale):
        if i == 0 or threshold != formatted_scale[i - 1][0]:
            filtered_scale.append((threshold, score))
    
    filtered_scale.sort(reverse=True, key=lambda x: x[0])  # Сортируем по границе убывания

    # Форматируем вывод с использованием имени переменной
    formatted_output = f"{scale_name} = [\n"
    for i, (threshold, score) in enumerate(filtered_scale):
        if i == 0:
            formatted_output += f"    ({threshold:.2f}, {int(score) if score.is_integer() else score:.6f}),  # Если значение больше {threshold:.2f}%, то {int(score) if score.is_integer() else score} баллов\n"
        else:
            formatted_output += f"    ({threshold:.2f}, {int(score) if score.is_integer() else score:.6f}),  # Если значение в диапазоне {threshold:.2f}% – {filtered_scale[i-1][0]:.2f}%, то {int(score) if score.is_integer() else score} баллов\n"
    
    # Добавляем последнее значение
    formatted_output += f"    (0, 10)        # Если значение меньше или равно {filtered_scale[-1][0]:.2f}%, то 10 баллов\n"
    formatted_output += "]"
    return formatted_output



In [19]:
def format_scale(scale_df, scale_name):
    """
    Генерирует отформатированный список на основе DataFrame с указанием имени переменной.

    Args:
        scale_df (pd.DataFrame): DataFrame с диапазонами и баллами.
        scale_name (str): Имя переменной для итогового списка.

    Returns:
        str: Отформатированный список в виде строки.
    """
    formatted_scale = []
    ranges = scale_df['Процент выполнения плана (%)']
    scores = scale_df['Баллы за выполнение']
    
    # Смещаем баллы на одну позицию
    shifted_scores = list(scores[1:]) + [scores.iloc[-1]]  # Последнее значение дублируется для крайних случаев
    
    for i in range(len(ranges)):
        range_str = ranges[i]
        score = shifted_scores[i]  # Используем сдвинутый список
        
        # Извлекаем диапазоны из строк формата "x% – y%" или "> x%"
        if " – " in range_str:
            lower, upper = map(lambda x: float(x.strip('%')), range_str.split(" – "))
            if lower != upper:  # Пропускаем одинаковые нижнюю и верхнюю границы
                formatted_scale.append((upper, score))
        elif "> " in range_str:
            lower = float(range_str.split("> ")[1].strip('%'))
            formatted_scale.append((lower, score))  # Больше, чем нижняя граница
        elif "≤ " in range_str:
            upper = float(range_str.split("≤ ")[1].strip('%'))
            formatted_scale.append((upper, score))  # Меньше или равно верхней границе

    # Удаляем дубликаты значений с одинаковой границей и сортируем
    filtered_scale = []
    for i, (threshold, score) in enumerate(formatted_scale):
        if i == 0 or threshold != formatted_scale[i - 1][0]:
            filtered_scale.append((threshold, score))
    
    filtered_scale.sort(reverse=True, key=lambda x: x[0])  # Сортируем по границе убывания

    # Форматируем вывод с использованием имени переменной
    formatted_output = f"{scale_name} = [\n"
    for i, (threshold, score) in enumerate(filtered_scale):
        if isinstance(score, float):  # Проверяем тип данных
            score_str = f"{int(score) if score.is_integer() else score:.6f}"
        else:
            score_str = str(score)  # Для целых чисел
        if i == 0:
            formatted_output += f"    ({threshold:.2f}, {score_str}),  # Если значение больше {threshold:.2f}%, то {score} баллов\n"
        else:
            formatted_output += f"    ({threshold:.2f}, {score_str}),  # Если значение в диапазоне {threshold:.2f}% – {filtered_scale[i-1][0]:.2f}%, то {score} баллов\n"
    
    # Добавляем последнее значение
    formatted_output += f"    (0, 10)        # Если значение меньше или равно {filtered_scale[-1][0]:.2f}%, то 10 баллов\n"
    formatted_output += "]"
    return formatted_output

In [21]:
def adjust_scores0(scale_df, min_score=10, max_score=80):
    """
    Функция для корректировки баллов в DataFrame, созданном build_scale.

    :param scale_df: DataFrame с диапазонами процентов и баллами
    :param min_score: Минимальный балл для значений до 100%
    :param max_score: Максимальный балл для значений выше 100%
    :return: DataFrame с откорректированными баллами
    """
    def extract_upper_bound(range_str):
        """
        Извлекает верхнюю границу диапазона из строки формата:
        '≤ 50.00%', '50.00% – 100.00%', '> 100.00%'
        """
        if '≤' in range_str:
            return float(range_str.split('≤')[1].strip('% '))
        elif '>' in range_str:
            return float(range_str.split('>')[1].strip('% '))
        else:  # Формат 'x% – y%'
            return float(range_str.split('–')[1].strip('% '))

    # Извлекаем верхние границы диапазонов
    upper_bounds = scale_df['Процент выполнения плана (%)'].apply(extract_upper_bound)

    # Определяем диапазоны ниже и выше 100%
    below_100_indices = upper_bounds <= 100
    above_100_indices = upper_bounds > 100

    # Количество диапазонов ниже и выше 100%
    below_100_count = below_100_indices.sum()
    above_100_count = above_100_indices.sum()

    # Генерация равномерных баллов
    below_100_scores = np.linspace(min_score, 50, below_100_count) if below_100_count > 0 else []
    above_100_scores = np.linspace(50, max_score, above_100_count) if above_100_count > 0 else []

    # Обновляем баллы в DataFrame
    scale_df.loc[below_100_indices, 'Баллы за выполнение'] = below_100_scores
    scale_df.loc[above_100_indices, 'Баллы за выполнение'] = above_100_scores

    return scale_df

In [23]:
def adjust_scores(scale_df, min_score=10, max_score=80):
    """
    Корректировка баллов в DataFrame: строго один интервал с 100% получает 50 баллов,
    остальные распределяются равномерно ниже и выше.
    
    :param scale_df: DataFrame с диапазонами процентов и баллами
    :param min_score: Минимальный балл для значений до 100%
    :param max_score: Максимальный балл для значений выше 100%
    :return: DataFrame с откорректированными баллами
    """
    def extract_upper_bound(range_str):
        """
        Извлекает верхнюю границу диапазона из строки формата:
        '≤ 50.00%', '50.00% – 100.00%', '> 100.00%'
        """
        if '≤' in range_str:
            return float(range_str.split('≤')[1].strip('% '))
        elif '>' in range_str:
            return float(range_str.split('>')[1].strip('% '))
        else:  # Формат 'x% – y%'
            return float(range_str.split('–')[1].strip('% '))

    def extract_lower_bound(range_str):
        """
        Извлекает нижнюю границу диапазона из строки формата:
        '≤ 50.00%', '50.00% – 100.00%', '> 100.00%'
        """
        if '≤' in range_str or '>' in range_str:
            return 0  # Для первых и последних интервалов
        else:
            return float(range_str.split('–')[0].strip('% '))

    # Извлекаем границы диапазонов
    scale_df['lower_bound'] = scale_df['Процент выполнения плана (%)'].apply(extract_lower_bound)
    scale_df['upper_bound'] = scale_df['Процент выполнения плана (%)'].apply(extract_upper_bound)

    # Определяем минимальный интервал, содержащий 100%
    containing_100 = scale_df[(scale_df['lower_bound'] <= 100) & (scale_df['upper_bound'] >= 100)].index
    if containing_100.empty:
        raise ValueError("Не удалось найти интервал, содержащий 100%")
    containing_100_index = containing_100.min()  # Берем минимальный индекс

    # Деление на интервалы выше и ниже 100%
    below_100_indices = scale_df.loc[:containing_100_index].index
    above_100_indices = scale_df.loc[containing_100_index + 1:].index

    # Генерация равномерных баллов для интервалов ниже 100%
    if len(below_100_indices) > 1:
        below_100_scores = np.linspace(min_score, 50, len(below_100_indices)).tolist()
    else:
        below_100_scores = [50]

    # Генерация равномерных баллов для интервалов выше 100%
    if len(above_100_indices) > 0:
        above_100_scores = np.linspace(50 + (max_score - 50) / len(above_100_indices), max_score, len(above_100_indices)).tolist()
    else:
        above_100_scores = []

    # Обновляем баллы в DataFrame
    scale_df.loc[below_100_indices, 'Баллы за выполнение'] = below_100_scores
    scale_df.loc[containing_100_index, 'Баллы за выполнение'] = 50  # Устанавливаем 50 баллов для 100%
    scale_df.loc[above_100_indices, 'Баллы за выполнение'] = above_100_scores

    # Удаляем временные столбцы
    scale_df.drop(['lower_bound', 'upper_bound'], axis=1, inplace=True)

    return scale_df

In [26]:
# Сохраняем DataFrame в CSV файл
#df.to_csv('dannie6.csv', index=False)

# Очищаем DataFrame
df = pd.DataFrame()

# Загружаем данные из CSV файла
df = pd.read_csv('dannie.csv')

In [28]:
df

,kafedra,DolMagFact,DolMagVip,EGEFact,EGEVip,DolStorMagFact,DolStorMagVip,DolCelevikovFact,DolCelevikovVip,ObemPOUFact,...,ChlenstvoKomitetovFact,ChlenstvoKomitetovVip,InnovaciiFact,InnovaciiVip,DolPPS200Fact,DolPPS200Vip,DolMolNPRFact,DolMolNPRVip,RIDFact,RIDVip
0,АСУТП,25.082508,127.052551,64.000000,88.593577,7.894737,46.650718,20.289914,118.251837,174.832215,...,7.0,0.0,4.0,133.333333,20.69,53.051282,0,0,10.0,333.333333
1,АЭП,30.833333,108.311966,63.631579,88.083581,21.621622,84.324324,0.900909,25.593801,173.628607,...,6.0,0.0,3.0,150.000000,63.16,126.320000,0,0,8.0,266.666667
2,АЭС,27.007299,88.981944,72.159091,99.888000,21.621622,66.259808,16.412276,69.347416,152.998882,...,6.0,0.0,0.0,0.000000,19.23,64.100000,0,0,6.0,150.000000
3,БИТ,12.504643,178.922687,81.088542,112.248812,22.222222,119.191919,2.450982,130.651438,2538.014807,...,3.0,0.0,3.0,50.000000,37.71,90.000000,0,0,0.0,0.000000
4,ВМ,0.000000,0.000000,0.001000,0.000000,0.000000,0.000000,0.000000,0.000000,202.108311,...,10.0,0.0,3.0,30.000000,23.64,76.480104,0,0,0.0,0.000000
5,ВМСС,16.561965,93.373671,79.671053,110.286618,21.551724,113.916256,2.072143,81.529325,288.355097,...,16.0,0.0,4.0,66.666667,33.33,109.242871,0,0,0.0,0.000000
6,ВТ,14.646465,86.722488,79.954248,110.678638,34.482759,87.356322,7.731999,141.859170,131.452263,...,12.0,0.0,2.0,100.000000,22.22,74.066667,0,0,2.0,100.000000
7,ГВИЭ,28.622482,89.197131,63.568627,87.996439,26.966292,71.653291,2.917780,46.070065,184.128218,...,11.0,0.0,1.0,33.333333,30.43,60.860000,0,0,5.0,83.333333
8,ГГМ,44.329897,144.482627,61.066667,84.533038,23.255814,104.651163,2.380981,175.324598,66.609243,...,3.0,0.0,0.0,0.000000,30.00,61.893955,0,0,11.0,1100.000000
9,Дизайн,0.000000,0.000000,0.001000,0.000000,0.000000,0.000000,0.000000,0.000000,3855.941378,...,1.0,0.0,2.0,200.000000,86.67,173.340000,0,0,0.0,0.000000


In [30]:
# Отфильтровать DataFrame, оставив только ненулевые значения в столбце sredFact
df_nonzero = df[df['EGEVip'] >= 0.002]

In [32]:
analyze_distribution(df_nonzero, 'EGEVip')

Минимум: 77.5809031622985
Максимум: 119.308802941975
Медиана (Q2): 91.3100298884244
Q1 (25-й перцентиль): 87.9964388856318
Q3 (75-й перцентиль): 99.8879995973019
Межквартильный размах (IQR): 11.891560711670095
Тип скошенности: правостороннее скошенное распределение
Выбросы: 112.248811830934, 110.286617707058, 110.678638380417, 106.751050753499, 112.764097887217, 105.705567729318, 119.308802941975, 107.872747407631


In [34]:
result = sorted_column_to_string(df_nonzero, 'EGEVip')
print(result)

77.5809031622985; 79.1243359494118; 83.640045165353; 84.5330380214102; 85.2757123521243; 85.6958287190846; 85.6968541077068; 86.2332112332113; 86.4610457633714; 87.1656540261192; 87.7546226383436; 87.9964388856318; 88.0835810456373; 88.3299056056531; 88.59357696567; 88.6938867221981; 88.8952778487662; 89.7878531257464; 90.2854681924449; 90.4690458328868; 90.503661352769; 91.0435233524935; 91.3100298884244; 93.17286933566; 93.8728204460079; 93.8945976804279; 94.4243095405886; 94.5990916921149; 95.5270929261138; 95.5457118247816; 98.5911160329765; 98.9866230730018; 99.1176136524974; 99.8879995973019; 100.08647444048; 100.752122554448; 100.931951839884; 105.705567729318; 106.751050753499; 107.872747407631; 110.286617707058; 110.678638380417; 112.248811830934; 112.764097887217; 119.308802941975


In [36]:
#sredProcVip

In [38]:
# Пример использования функции с вариантом №1 и 4 секциями (квартилями)
scale_variant1_quartiles = build_scale_quartiles(df_nonzero['EGEVip'], num_sections=10,min_score=10, max_score=80)
print("Шкала с использованием квартилей (10 секции):")
print("Процент выполнения плана (%); Баллы за выполнение;") 
# Форматирование вывода
output = "\n".join(f"{row['Процент выполнения плана (%)']}; {row['Баллы за выполнение']};" for _, row in scale_variant1_quartiles.iterrows())
print(output)




Шкала с использованием квартилей (10 секции):
Процент выполнения плана (%); Баллы за выполнение;
≤ 85.44%; 10;
85.44% – 87.02%; 17;
87.02% – 88.38%; 25;
88.38% – 90.09%; 33;
90.09% – 91.31%; 41;
91.31% – 94.49%; 48;
94.49% – 98.91%; 56;
98.91% – 100.79%; 64;
100.79% – 109.32%; 72;
> 109.32%; 80;


In [40]:
def extract_upper_bound(range_str):
        """
        Извлекает верхнюю границу диапазона из строки формата:
        '≤ 50.00%', '50.00% – 100.00%', '> 100.00%'
        """
        if '≤' in range_str:
            return float(range_str.split('≤')[1].strip('% '))
        elif '>' in range_str:
            return float(range_str.split('>')[1].strip('% '))
        else:  # Формат 'x% – y%'
            return float(range_str.split('–')[1].strip('% '))

def extract_lower_bound(range_str):
        """
        Извлекает нижнюю границу диапазона из строки формата:
        '≤ 50.00%', '50.00% – 100.00%', '> 100.00%'
        """
        if '≤' in range_str or '>' in range_str:
            return 0  # Для первых и последних интервалов
        else:
            return float(range_str.split('–')[0].strip('% '))

In [42]:
scale_variant1_quartiles

,Процент выполнения плана (%),Баллы за выполнение
0,≤ 85.44%,10
1,85.44% – 87.02%,17
2,87.02% – 88.38%,25
3,88.38% – 90.09%,33
4,90.09% – 91.31%,41
5,91.31% – 94.49%,48
6,94.49% – 98.91%,56
7,98.91% – 100.79%,64
8,100.79% – 109.32%,72
9,> 109.32%,80


In [44]:
scale_df = scale_variant1_quartiles.copy()

In [46]:
scale_df

,Процент выполнения плана (%),Баллы за выполнение
0,≤ 85.44%,10
1,85.44% – 87.02%,17
2,87.02% – 88.38%,25
3,88.38% – 90.09%,33
4,90.09% – 91.31%,41
5,91.31% – 94.49%,48
6,94.49% – 98.91%,56
7,98.91% – 100.79%,64
8,100.79% – 109.32%,72
9,> 109.32%,80


In [48]:
scale_df['lower_bound'] = scale_df['Процент выполнения плана (%)'].apply(extract_lower_bound)
scale_df['upper_bound'] = scale_df['Процент выполнения плана (%)'].apply(extract_upper_bound)

In [50]:
scale_df['lower_bound'] = scale_df['lower_bound'].astype(float)
scale_df['upper_bound'] = scale_df['upper_bound'].astype(float)

In [52]:
scale_df

,Процент выполнения плана (%),Баллы за выполнение,lower_bound,upper_bound
0,≤ 85.44%,10,0.00,85.44
1,85.44% – 87.02%,17,85.44,87.02
2,87.02% – 88.38%,25,87.02,88.38
3,88.38% – 90.09%,33,88.38,90.09
4,90.09% – 91.31%,41,90.09,91.31
5,91.31% – 94.49%,48,91.31,94.49
6,94.49% – 98.91%,56,94.49,98.91
7,98.91% – 100.79%,64,98.91,100.79
8,100.79% – 109.32%,72,100.79,109.32
9,> 109.32%,80,0.00,109.32


In [54]:
containing_100 = scale_df[(scale_df['lower_bound'] <= 100) & (scale_df['upper_bound'] >= 100)].index

In [56]:
containing_100_index = containing_100.min()  # Берем минимальный индекс

In [58]:
containing_100_index

7

In [60]:
below_100_indices = scale_df.loc[:containing_100_index].index

In [62]:
above_100_indices = scale_df.loc[containing_100_index + 1:].index

In [64]:
below_100_indices

RangeIndex(start=0, stop=8, step=1)

In [66]:
above_100_indices

RangeIndex(start=8, stop=10, step=1)

In [68]:
max_score = 80
# Генерация равномерных баллов для интервалов ниже 100%
if len(below_100_indices) > 1:
    below_100_scores = np.linspace(10, 50, len(below_100_indices)).tolist()
else:
    below_100_scores = [50]

# Генерация равномерных баллов для интервалов выше 100%
if len(above_100_indices) > 0:
    above_100_scores = np.linspace(50 + (max_score - 50) / len(above_100_indices), max_score, len(above_100_indices)).tolist()
else:
    above_100_scores = []

In [70]:
# Обновляем баллы в DataFrame
scale_df.loc[below_100_indices, 'Баллы за выполнение'] = below_100_scores
scale_df.loc[containing_100_index, 'Баллы за выполнение'] = 50  # Устанавливаем 50 баллов для 100%
scale_df.loc[above_100_indices, 'Баллы за выполнение'] = above_100_scores

# Удаляем временные столбцы
scale_df.drop(['lower_bound', 'upper_bound'], axis=1, inplace=True)


C:\Temp\ipykernel_4256\1590286579.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[10.0, 15.714285714285715, 21.42857142857143, 27.142857142857142, 32.85714285714286, 38.57142857142857, 44.285714285714285, 50.0]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  scale_df.loc[below_100_indices, 'Баллы за выполнение'] = below_100_scores


In [72]:
below_100_scores

[10.0,
 15.714285714285715,
 21.42857142857143,
 27.142857142857142,
 32.85714285714286,
 38.57142857142857,
 44.285714285714285,
 50.0]

In [74]:
above_100_scores

[65.0, 80.0]

In [76]:
above_100_indices

RangeIndex(start=8, stop=10, step=1)

In [78]:
scale_df

,Процент выполнения плана (%),Баллы за выполнение
0,≤ 85.44%,10.000000
1,85.44% – 87.02%,15.714286
2,87.02% – 88.38%,21.428571
3,88.38% – 90.09%,27.142857
4,90.09% – 91.31%,32.857143
5,91.31% – 94.49%,38.571429
6,94.49% – 98.91%,44.285714
7,98.91% – 100.79%,50.000000
8,100.79% – 109.32%,65.000000
9,> 109.32%,80.000000


In [81]:
adjusted_scale_df = adjust_scores(scale_variant1_quartiles, min_score=10, max_score=80)
print(adjusted_scale_df)
formatted_scale = format_scale(adjusted_scale_df, "scaleEgeVip")
print(formatted_scale)

  Процент выполнения плана (%)  Баллы за выполнение
0                     ≤ 85.44%            10.000000
1              85.44% – 87.02%            15.714286
2              87.02% – 88.38%            21.428571
3              88.38% – 90.09%            27.142857
4              90.09% – 91.31%            32.857143
5              91.31% – 94.49%            38.571429
6              94.49% – 98.91%            44.285714
7             98.91% – 100.79%            50.000000
8            100.79% – 109.32%            65.000000
9                    > 109.32%            80.000000
scaleEgeVip = [
    (109.32, 80.000000),  # Если значение больше 109.32%, то 80.0 баллов
    (100.79, 65.000000),  # Если значение в диапазоне 100.79% – 109.32%, то 65.0 баллов
    (98.91, 50.000000),  # Если значение в диапазоне 98.91% – 100.79%, то 50.0 баллов
    (94.49, 44.285714),  # Если значение в диапазоне 94.49% – 98.91%, то 44.285714285714285 баллов
    (91.31, 38.571429),  # Если значение в диапазоне 91.31% – 94.4

C:\Temp\ipykernel_4256\2717947918.py:60: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[10.0, 15.714285714285715, 21.42857142857143, 27.142857142857142, 32.85714285714286, 38.57142857142857, 44.285714285714285, 50.0]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  scale_df.loc[below_100_indices, 'Баллы за выполнение'] = below_100_scores


In [83]:
fields = df.columns.tolist()
print(fields)

['kafedra', 'DolMagFact', 'DolMagVip', 'EGEFact', 'EGEVip', 'DolStorMagFact', 'DolStorMagVip', 'DolCelevikovFact', 'DolCelevikovVip', 'ObemPOUFact', 'ObemPOUVip', 'DolOstepenennykhFact', 'DolOstepenennykhVip', 'TrudoustroistvoFact', 'TrudoustroistvoVip', 'KolPublikFact', 'KolPublikVip', 'ObemNIOKRFact', 'ObemNIOKRVip', 'ZashchityFact', 'ZashchityVip', 'DolInostrFact', 'DolInostrVip', 'KolInostrPrepodFact', 'KolInostrPrepodVip', 'AkadMobFact', 'AkadMobVip', 'ZayavAktivFact', 'ZayavAktivVip', 'PublNauchMaterialFact', 'PublNauchMaterialVip', 'PodgotovkaEnciklFact', 'PodgotovkaEnciklVip', 'RukovRabotyFact', 'RukovRabotyVip', 'UchastieVystavkiFact', 'UchastieVystavkiVip', 'ChlenstvoSovetovFact', 'ChlenstvoSovetovVip', 'ChlenstvoKomitetovFact', 'ChlenstvoKomitetovVip', 'InnovaciiFact', 'InnovaciiVip', 'DolPPS200Fact', 'DolPPS200Vip', 'DolMolNPRFact', 'DolMolNPRVip', 'RIDFact', 'RIDVip']


In [85]:
#trudProcVip

In [277]:
df_nonzero = df[df['TrudoustroistvoVip'] >= 0.002]

In [61]:
analyze_distribution(df_nonzero, 'TrudoustroistvoVip')

Минимум: 97.0
Максимум: 100.0
Медиана (Q2): 100.0
Q1 (25-й перцентиль): 100.0
Q3 (75-й перцентиль): 100.0
Межквартильный размах (IQR): 0.0
Тип скошенности: правостороннее скошенное распределение
Выбросы: Нет выбросов


In [62]:
result = sorted_column_to_string(df_nonzero, 'TrudoustroistvoVip')
print(result)

97.0; 97.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0


In [ ]:
#ridProcVip

In [321]:
df_nonzero = df[df['RIDVip'] >= 0.002]

In [322]:
result = sorted_column_to_string(df_nonzero, 'RIDVip')
print(result)

23.5294117647059; 33.3333333333333; 50.0; 60.0; 63.1578947368421; 66.6666666666667; 83.3333333333333; 88.2352941176471; 100.0; 100.0; 100.0; 100.0; 100.0; 112.5; 125.0; 140.0; 150.0; 150.0; 150.0; 200.0; 200.0; 200.0; 200.0; 200.0; 230.0; 233.333333333333; 250.0; 266.666666666667; 300.0; 333.333333333333; 400.0; 400.0; 500.0; 600.0; 650.0; 1100.0; 1400.0


In [323]:
analyze_distribution(df_nonzero, 'RIDVip')

Минимум: 23.5294117647059
Максимум: 1400.0
Медиана (Q2): 150.0
Q1 (25-й перцентиль): 100.0
Q3 (75-й перцентиль): 266.666666666667
Межквартильный размах (IQR): 166.66666666666703
Тип скошенности: правостороннее скошенное распределение
Выбросы: 333.333333333333, 266.666666666667, 150.0, 1100.0, 600.0, 150.0, 200.0, 300.0, 200.0, 150.0, 125.0, 500.0, 233.333333333333, 200.0, 200.0, 250.0, 650.0, 112.5, 400.0, 1400.0, 400.0, 200.0, 140.0, 230.0


In [325]:
#build_scale(percentages, num_grades=10, min_score=50, max_score=80, scale_type='linear')
scale_variant3 = build_scale(df_nonzero['RIDVip'],num_grades=20, min_score=10, max_score=90, scale_type='log')
adjusted_scale_df = adjust_scores(scale_variant3, min_score=10, max_score=90)
print(adjusted_scale_df)
formatted_scale = format_scale(adjusted_scale_df,"scaleRidVip")
print(formatted_scale)

   Процент выполнения плана (%)  Баллы за выполнение
0                      ≤ 36.90%            10.000000
1               36.90% – 54.78%            16.666667
2               54.78% – 62.35%            23.333333
3               62.35% – 66.67%            30.000000
4               66.67% – 84.53%            36.666667
5               84.53% – 93.94%            43.333333
6              93.94% – 109.24%            50.000000
7             109.24% – 125.00%            53.076923
8             125.00% – 142.44%            56.153846
9             142.44% – 173.22%            59.230769
10            173.22% – 222.10%            62.307692
11            222.10% – 233.33%            65.384615
12            233.33% – 254.07%            68.461538
13            254.07% – 282.84%            71.538462
14            282.84% – 324.67%            74.615385
15            324.67% – 400.00%            77.692308
16            400.00% – 523.32%            80.769231
17            523.32% – 624.50%            83.

In [308]:
#def build_scale(percentages, num_grades=10, min_score=10, max_score=100, scale_type='linear'):
#    """
#    Функция для построения шкалы присвоения баллов на основе распределения данных и заданных критериев,
#    с удалением одинаковых интервалов.

#    :param percentages: список процентных значений выполнения плана
#    :param num_grades: количество градаций шкалы (по умолчанию 10)
#    :param min_score: минимальный балл (по умолчанию 10)
#    :param max_score: максимальный балл (по умолчанию 100)
#    :param scale_type: тип масштабирования ('linear' или 'log')
#    :return: DataFrame с диапазонами процентов и соответствующими баллами

df_unique = list(set(df_nonzero['RIDVip'].tolist()))
# Преобразуем список в numpy array
data = np.array(df_unique)


# Проверяем максимальное значение
max_percent = max(data)

# Обрабатываем данные в зависимости от выбранного масштабирования
data = np.log(data + 1)  # Логарифмическое масштабирование
    
 

  

In [309]:
max_percent

1400.0

In [310]:
# Сортируем данные
data_sorted = np.sort(data)
data_sorted

array([3.19987288, 3.5361167 , 3.93182563, 4.11087386, 4.16134715,
       4.21459369, 4.4347772 , 4.49127664, 4.61512052, 4.73180284,
       4.83628191, 4.94875989, 5.01727984, 5.30330491, 5.44241771,
       5.4567446 , 5.52545294, 5.58974243, 5.70711026, 5.8121385 ,
       5.99396143, 6.2166061 , 6.39859493, 6.47850964, 7.00397414,
       7.24494155])

In [311]:
num_grades = 20

In [312]:
# Разделяем данные на перцентили
percentiles = np.linspace(0, 100, num_grades + 1)  # Делим на num_grades диапазонов
percentiles

array([  0.,   5.,  10.,  15.,  20.,  25.,  30.,  35.,  40.,  45.,  50.,
        55.,  60.,  65.,  70.,  75.,  80.,  85.,  90.,  95., 100.])

In [313]:
bins = np.percentile(data_sorted, percentiles)
bins

array([3.19987288, 3.63504393, 4.02134975, 4.14872883, 4.21459369,
       4.44890206, 4.55319858, 4.70263226, 4.83628191, 4.96588988,
       5.16029237, 5.40763951, 5.4567446 , 5.54152531, 5.64842635,
       5.78588144, 5.99396143, 6.26210331, 6.43855229, 6.87260801,
       7.24494155])

In [317]:
# Создаем диапазоны процентов
ranges = []
scores = []
min_score = 10
max_score = 90
for i in range(len(bins) - 1):
        lower = bins[i]
        upper = bins[i + 1]
        lower_display = np.exp(lower) - 1
        upper_display = np.exp(upper) - 1
        if i == 0:
            range_str = f"≤ {upper_display:.2f}%"
        elif i == len(bins) - 2:
            range_str = f"> {lower_display:.2f}%"
        else:
            range_str = f"{lower_display:.2f}% – {upper_display:.2f}%"
        # Пропускаем повторяющиеся диапазоны
        #if ranges and ranges[-1][1] == upper_display:
        #    continue

        ranges.append((lower_display, upper_display, range_str))
        # Присваиваем баллы
        if upper <= 100:  # Невыполнение плана (до 100%)
            score = min_score + (50 - min_score) * (upper / 100)
        else:  # От 100% равномерное распределение
            relative_position = (upper - 100) / (max_percent - 100)
            score = 50 + (max_score - 50) * relative_position

        scores.append(round(score))  # Округляем до целого


In [318]:
ranges

[(23.529411764705902, 36.90351807022217, '≤ 36.90%'),
 (36.90351807022217, 54.77633906953735, '36.90% – 54.78%'),
 (54.77633906953735, 62.353416051578726, '54.78% – 62.35%'),
 (62.353416051578726, 66.6666666666667, '62.35% – 66.67%'),
 (66.6666666666667, 84.53298225001731, '66.67% – 84.53%'),
 (84.53298225001731, 93.93558187467099, '84.53% – 93.94%'),
 (93.93558187467099, 109.23696288971615, '93.94% – 109.24%'),
 (109.23696288971615, 125.0, '109.24% – 125.00%'),
 (125.0, 142.4361340128213, '125.00% – 142.44%'),
 (142.4361340128213, 173.21538393609222, '142.44% – 173.22%'),
 (173.21538393609222, 222.1043300711476, '173.22% – 222.10%'),
 (222.1043300711476, 233.3333333333331, '222.10% – 233.33%'),
 (233.3333333333331, 254.06675894228212, '233.33% – 254.07%'),
 (254.06675894228212, 282.84444096488255, '254.07% – 282.84%'),
 (282.84444096488255, 324.6689715099585, '282.84% – 324.67%'),
 (324.6689715099585, 400.00000000000006, '324.67% – 400.00%'),
 (400.00000000000006, 523.3205895965702, '

In [319]:
scores

[11,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 13,
 13,
 13,
 13]

In [ ]:


    
    

    # Создаем диапазоны процентов
    ranges = []
    scores = []
    for i in range(len(bins) - 1):
        lower = bins[i]
        upper = bins[i + 1]

        # Преобразуем границы обратно, если использовали логарифмическое масштабирование
        if scale_type == 'log':
            lower_display = np.exp(lower) - 1
            upper_display = np.exp(upper) - 1
        else:
            lower_display = lower
            upper_display = upper

        if i == 0:
            range_str = f"≤ {upper_display:.2f}%"
        elif i == len(bins) - 2:
            range_str = f"> {lower_display:.2f}%"
        else:
            range_str = f"{lower_display:.2f}% – {upper_display:.2f}%"

        # Пропускаем повторяющиеся диапазоны
        if ranges and ranges[-1][1] == upper_display:
            continue

        ranges.append((lower_display, upper_display, range_str))

        # Присваиваем баллы
        if upper <= 100:  # Невыполнение плана (до 100%)
            score = min_score + (50 - min_score) * (upper / 100)
        else:  # От 100% равномерное распределение
            relative_position = (upper - 100) / (max_percent - 100)
            score = 50 + (max_score - 50) * relative_position

        scores.append(round(score))  # Округляем до целого

    # Создаем DataFrame для шкалы
    scale_df = pd.DataFrame({
        'Процент выполнения плана (%)': [r[2] for r in ranges],
        'Баллы за выполнение': scores
    })

    return scale_df

In [139]:
scale_variant3 = build_scale(df_nonzero['RIDVip'],num_grades=10, min_score=10, max_score=80, scale_type='log')

In [140]:
scale_variant3

,Процент выполнения плана (%),Баллы за выполнение
0,≤ 61.88%,12
1,61.88% – 90.47%,12
2,90.47% – 100.00%,12
3,100.00% – 130.80%,12
4,130.80% – 150.00%,12
5,150.00% – 200.00%,12
6,200.00% – 236.58%,12
7,236.58% – 326.38%,12
8,326.38% – 537.83%,13
9,> 537.83%,13


In [184]:
print("Процент выполнения плана (%); Баллы за выполнение;") 
# Форматирование вывода
output = "\n".join(f"{row['Процент выполнения плана (%)']}; {row['Баллы за выполнение']};" for _, row in scale_variant3.iterrows())
print(output)


Процент выполнения плана (%); Баллы за выполнение;
≤ 50.00%; 30;
50.00% – 86.43%; 45;
86.43% – 100.00%; 60;
100.00% – 100.00%; 60;
100.00% – 126.49%; 80;
126.49% – 150.00%; 80;
150.00% – 190.49%; 80;
190.49% – 219.76%; 80;
219.76% – 350.00%; 80;
> 350.00%; 80;


In [326]:
#PublNauchMaterialVip

In [327]:
result = sorted_column_to_string(df, 'PublNauchMaterialVip')
print(result)

0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 25.0; 25.0; 25.0; 25.0; 25.0; 50.0; 50.0; 50.0; 50.0; 50.0; 75.0; 75.0; 75.0; 75.0; 100.0; 100.0; 100.0; 100.0; 125.0; 150.0; 150.0; 200.0; 200.0; 200.0; 225.0; 250.0; 300.0; 300.0; 300.0; 325.0; 375.0; 375.0; 500.0; 525.0; 625.0; 650.0; 775.0; 800.0; 1075.0; 1125.0; 1225.0; 1700.0; 2175.0


In [328]:
# Отфильтровать DataFrame, оставив только ненулевые значения в столбце sredFact
df_nonzero = df[df['PublNauchMaterialVip'] >= 0.002]

In [329]:
analyze_distribution(df_nonzero, 'PublNauchMaterialVip')

Минимум: 25.0
Максимум: 2175.0
Медиана (Q2): 200.0
Q1 (25-й перцентиль): 75.0
Q3 (75-й перцентиль): 437.5
Межквартильный размах (IQR): 362.5
Тип скошенности: правостороннее скошенное распределение
Выбросы: 200.0, 300.0, 300.0, 150.0, 800.0, 300.0, 375.0, 650.0, 525.0, 2175.0, 1125.0, 775.0, 150.0, 200.0, 375.0, 500.0, 225.0, 250.0, 125.0, 1075.0, 325.0, 1700.0, 1225.0, 200.0, 625.0


In [330]:
scale_variant3 = build_scale(df_nonzero['PublNauchMaterialVip'],num_grades=20, min_score=10, max_score=90, scale_type='log')
adjusted_scale_df = adjust_scores(scale_variant3, min_score=10, max_score=90)
print(adjusted_scale_df)
formatted_scale = format_scale(adjusted_scale_df, "scalePublNauchMaterialVip")
print(formatted_scale)

   Процент выполнения плана (%)  Баллы за выполнение
0                      ≤ 52.08%            10.000000
1               52.08% – 79.45%            30.000000
2              79.45% – 106.93%            50.000000
3             106.93% – 134.46%            52.352941
4             134.46% – 173.22%            54.705882
5             173.22% – 214.65%            57.058824
6             214.65% – 242.22%            59.411765
7             242.22% – 289.26%            61.764706
8             289.26% – 322.41%            64.117647
9             322.41% – 375.00%            66.470588
10            375.00% – 502.45%            68.823529
11            502.45% – 543.63%            71.176471
12            543.63% – 632.40%            73.529412
13            632.40% – 697.38%            75.882353
14            697.38% – 787.40%            78.235294
15            787.40% – 955.18%            80.588235
16           955.18% – 1109.76%            82.941176
17          1109.76% – 1204.31%            85.

In [ ]:
AkadMob

In [331]:
# Отфильтровать DataFrame, оставив только ненулевые значения в столбце sredFact

result = sorted_column_to_string(df, 'AkadMobVip')
print(result)
df_nonzero = df[df['AkadMobVip'] >= 0.002]
analyze_distribution(df_nonzero, 'AkadMobVip')


0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 33.3333333333333; 33.3333333333333; 33.3333333333333; 33.3333333333333; 33.3333333333333; 33.3333333333333; 33.3333333333333; 33.3333333333333; 33.3333333333333; 66.6666666666667; 66.6666666666667; 66.6666666666667; 66.6666666666667; 66.6666666666667; 100.0; 100.0; 100.0; 100.0; 100.0; 133.333333333333; 133.333333333333; 133.333333333333; 133.333333333333; 133.333333333333; 133.333333333333; 166.666666666667; 233.333333333333; 233.333333333333; 266.666666666667; 333.333333333333; 366.666666666667; 366.666666666667; 400.0; 500.0; 3066.66666666667
Минимум: 33.3333333333333
Максимум: 3066.66666666667
Медиана (Q2): 100.0
Q1 (25-й перцентиль): 50.0
Q3 (75-й перцентиль): 200.0
Межквартильный размах (IQR): 150.0
Тип скошенности: правостороннее скошенное распределение
Выбросы: 166.666666666667, 133.333333333333, 400.0, 233.333333333333, 133.333333333333, 233.333333333333, 266.666666666667, 3066.6666666666

In [332]:
scale_variant3 = build_scale(df_nonzero['AkadMobVip'],num_grades=20, min_score=10, max_score=90, scale_type='log')
print(scale_variant3)

   Процент выполнения плана (%)  Баллы за выполнение
0                      ≤ 48.86%                   12
1               48.86% – 69.43%                   12
2               69.43% – 86.79%                   12
3              86.79% – 105.93%                   12
4             105.93% – 124.09%                   12
5             124.09% – 142.57%                   12
6             142.57% – 161.18%                   12
7             161.18% – 190.69%                   12
8             190.69% – 229.44%                   12
9             229.44% – 249.45%                   12
10            249.45% – 269.66%                   12
11            269.66% – 304.88%                   12
12            304.88% – 338.13%                   12
13            338.13% – 356.33%                   12
14            356.33% – 374.73%                   12
15            374.73% – 393.10%                   12
16            393.10% – 432.50%                   12
17            432.50% – 488.97%               

In [333]:
scale_variant3 = build_scale(df_nonzero['AkadMobVip'],num_grades=20, min_score=10, max_score=90, scale_type='log')
adjusted_scale_df = adjust_scores(scale_variant3, min_score=10, max_score=90)
print(adjusted_scale_df)
formatted_scale = format_scale(adjusted_scale_df, "scaleAkadMobVip")
print(formatted_scale)

   Процент выполнения плана (%)  Баллы за выполнение
0                      ≤ 48.86%            10.000000
1               48.86% – 69.43%            23.333333
2               69.43% – 86.79%            36.666667
3              86.79% – 105.93%            50.000000
4             105.93% – 124.09%            52.500000
5             124.09% – 142.57%            55.000000
6             142.57% – 161.18%            57.500000
7             161.18% – 190.69%            60.000000
8             190.69% – 229.44%            62.500000
9             229.44% – 249.45%            65.000000
10            249.45% – 269.66%            67.500000
11            269.66% – 304.88%            70.000000
12            304.88% – 338.13%            72.500000
13            338.13% – 356.33%            75.000000
14            356.33% – 374.73%            77.500000
15            374.73% – 393.10%            80.000000
16            393.10% – 432.50%            82.500000
17            432.50% – 488.97%            85.

In [334]:
# DolInostrVip
# Отфильтровать DataFrame, оставив только ненулевые значения в столбце sredFact
result = sorted_column_to_string(df, 'DolInostrVip')
print(result)
df_nonzero = df[df['DolInostrVip'] >= 0.002]
analyze_distribution(df_nonzero, 'DolInostrVip')

0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 15.6888029228455; 25.1863860111283; 29.6678121420389; 35.630927408633; 37.3448540987808; 45.6461750728194; 46.6917697686928; 61.0081861266696; 62.8458498023715; 64.6464646464646; 64.9191299498046; 65.7697362849015; 67.2043010752688; 67.6083402521591; 70.0158620005948; 71.7703349282297; 75.4629629629629; 75.9549896357714; 80.9153318077804; 82.3813354786806; 82.9045168667807; 87.1212121212121; 89.0089365504915; 91.0825815405968; 92.2412430203022; 93.8104448742748; 102.685950413223; 102.699999484438; 103.228228228228; 104.597701149425; 106.559920019041; 118.247148097894; 122.632433783109; 128.995606726254; 129.933754162042; 138.715929369297; 140.749962275539; 146.990641335679; 154.851187739082; 162.399616262233; 162.495721480613; 175.324675324675; 200.451722190852; 233.320704458351; 242.433221432443
Минимум: 15.6888029228455
Максимум: 242.433221432443
Медиана (Q2): 89.0089365504915
Q1 (25-й перцентиль): 65.7697362849015
Q3 (75-й перцентиль): 128.995

In [335]:
scale_variant3 = build_scale(df_nonzero['DolInostrVip'],num_grades=10, min_score=10, max_score=80, scale_type='log')
print(scale_variant3)

  Процент выполнения плана (%)  Баллы за выполнение
0                     ≤ 40.47%                   11
1              40.47% – 64.28%                   12
2              64.28% – 68.08%                   12
3              68.08% – 78.89%                   12
4              78.89% – 89.01%                   12
5             89.01% – 102.69%                   12
6            102.69% – 115.81%                   12
7            115.81% – 139.12%                   12
8            139.12% – 162.46%                   12
9                    > 162.46%                   12


In [336]:
scale_variant3 = build_scale(df_nonzero['DolInostrVip'],num_grades=10, min_score=10, max_score=80, scale_type='log')
adjusted_scale_df = adjust_scores(scale_variant3, min_score=10, max_score=80)
print(adjusted_scale_df)
formatted_scale = format_scale(adjusted_scale_df, "scaleDolInostrVip")
print(formatted_scale)

  Процент выполнения плана (%)  Баллы за выполнение
0                     ≤ 40.47%                 10.0
1              40.47% – 64.28%                 18.0
2              64.28% – 68.08%                 26.0
3              68.08% – 78.89%                 34.0
4              78.89% – 89.01%                 42.0
5             89.01% – 102.69%                 50.0
6            102.69% – 115.81%                 57.5
7            115.81% – 139.12%                 65.0
8            139.12% – 162.46%                 72.5
9                    > 162.46%                 80.0
scaleDolInostrVip = [
    (162.46, 80.000000),  # Если значение больше 162.46%, то 80.0 баллов
    (139.12, 72.500000),  # Если значение в диапазоне 139.12% – 162.46%, то 72.5 баллов
    (115.81, 65.000000),  # Если значение в диапазоне 115.81% – 139.12%, то 65.0 баллов
    (102.69, 57.500000),  # Если значение в диапазоне 102.69% – 115.81%, то 57.5 баллов
    (89.01, 50.000000),  # Если значение в диапазоне 89.01% – 102.69%

In [337]:
#DolMag
# Отфильтровать DataFrame, оставив только ненулевые значения в столбце sredFact
result = sorted_column_to_string(df, 'DolMagVip')
print(result)
df_nonzero = df[df['DolMagVip'] >= 0.002]
analyze_distribution(df_nonzero, 'DolMagVip')

0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 77.7080307428593; 86.0050890585242; 86.7224880382775; 87.2929292929291; 88.9819439108721; 89.1971311159661; 89.9580660540915; 90.8256880733944; 91.6576320939686; 92.6666863230728; 93.1223731749864; 93.373670643802; 96.611842105263; 97.1842157445965; 98.9261493690488; 100.003448394772; 100.467801333731; 101.148413376257; 105.494473491416; 106.01750547046; 107.249626307923; 108.311965811966; 110.257638667092; 112.993760819537; 113.247863247863; 115.384615384616; 116.11588455607; 117.558052434457; 121.408625504189; 121.6453841323; 122.648867556622; 123.67389298893; 125.161812297735; 126.04690117253; 126.162350313171; 127.052551408987; 127.493403693931; 132.686682063125; 135.76617087513; 136.016602221092; 137.832085139929; 144.482626956854; 168.938835666913; 178.92268721426; 180.979756132405; 248.824824185313
Минимум: 77.7080307428593
Максимум: 248.824824185313
Медиана (Q2): 111.6256997433145
Q1 (25-й перцентиль): 94.18321350916725
Q3 (75-й перцентиль): 1

In [338]:
scale_variant3 = build_scale(df_nonzero['DolMagVip'],num_grades=10, min_score=10, max_score=80, scale_type='log')
print(scale_variant3)

  Процент выполнения плана (%)  Баллы за выполнение
0                     ≤ 89.09%                   12
1              89.09% – 92.67%                   12
2              92.67% – 98.05%                   12
3             98.05% – 105.49%                   12
4            105.49% – 111.62%                   12
5            111.62% – 117.56%                   12
6            117.56% – 124.42%                   12
7            124.42% – 127.49%                   12
8            127.49% – 141.12%                   12
9                    > 141.12%                   12


In [339]:
scale_variant3 = build_scale(df_nonzero['DolMagVip'],num_grades=10, min_score=10, max_score=80, scale_type='log')
adjusted_scale_df = adjust_scores(scale_variant3, min_score=10, max_score=80)
print(adjusted_scale_df)
formatted_scale = format_scale(adjusted_scale_df, "scaleDolMagVip")
print(formatted_scale)

  Процент выполнения плана (%)  Баллы за выполнение
0                     ≤ 89.09%            10.000000
1              89.09% – 92.67%            23.333333
2              92.67% – 98.05%            36.666667
3             98.05% – 105.49%            50.000000
4            105.49% – 111.62%            55.000000
5            111.62% – 117.56%            60.000000
6            117.56% – 124.42%            65.000000
7            124.42% – 127.49%            70.000000
8            127.49% – 141.12%            75.000000
9                    > 141.12%            80.000000
scaleDolMagVip = [
    (141.12, 80.000000),  # Если значение больше 141.12%, то 80.0 баллов
    (127.49, 75.000000),  # Если значение в диапазоне 127.49% – 141.12%, то 75.0 баллов
    (124.42, 70.000000),  # Если значение в диапазоне 124.42% – 127.49%, то 70.0 баллов
    (117.56, 65.000000),  # Если значение в диапазоне 117.56% – 124.42%, то 65.0 баллов
    (111.62, 60.000000),  # Если значение в диапазоне 111.62% – 117.56%,

In [340]:
#DolOstepenennykh
# Отфильтровать DataFrame, оставив только ненулевые значения в столбце sredFact
result = sorted_column_to_string(df, 'DolOstepenennykhVip')
print(result)
df_nonzero = df[df['DolOstepenennykhVip'] >= 0.002]
analyze_distribution(df_nonzero, 'DolOstepenennykhVip')

11.2314480545527; 39.039039039039; 40.3628117913832; 55.2402572833901; 56.8222946073109; 57.2089947089947; 61.7702448210923; 61.8317061831706; 65.0988411724608; 66.7848699763593; 68.5358255451713; 69.5443645083933; 69.7399527186761; 70.3871292106586; 72.5010725010725; 72.7367870225013; 73.5524256651017; 74.2496050552923; 74.6839654025283; 75.0487329434698; 75.3968253968254; 75.5885997521685; 77.7777777777778; 79.56536978619; 80.0963565191207; 80.2919708029197; 80.7291666666667; 80.8580858085809; 82.0283370618941; 83.3333333333333; 83.5222978080121; 85.7898715041572; 85.8262108262108; 86.9565217391304; 87.8136200716846; 89.7435897435898; 90.1960784313725; 92.2705314009662; 92.3845193508115; 92.8903179707038; 93.5033394049787; 95.7854406130268; 96.5327029156816; 97.9406130268199; 98.9583333333333; 99.1452991452991; 100.39481105471; 100.580973129993; 101.205857019811; 101.941747572816; 104.384653712008; 105.520614954577; 108.206245461147
Минимум: 11.2314480545527
Максимум: 108.20624546114

In [342]:
# Пример использования функции с вариантом №1 и 4 секциями (квартилями)
scale_variant1_quartiles = build_scale_quartiles(df_nonzero['DolOstepenennykhVip'], num_sections=20,min_score=10, max_score=80)
print("Шкала с использованием квартилей (10 секции):")
print("Процент выполнения плана (%); Баллы за выполнение;") 
# Форматирование вывода
output = "\n".join(f"{row['Процент выполнения плана (%)']}; {row['Баллы за выполнение']};" for _, row in scale_variant1_quartiles.iterrows())
print(output)
adjusted_scale_df = adjust_scores(scale_variant1_quartiles, min_score=10, max_score=80)
print(adjusted_scale_df)
formatted_scale = format_scale(adjusted_scale_df, "scaleDolOstepenennykhVip")
print(formatted_scale)


Шкала с использованием квартилей (10 секции):
Процент выполнения плана (%); Баллы за выполнение;
≤ 49.29%; 10;
49.29% – 58.12%; 13;
58.12% – 64.45%; 17;
64.45% – 68.94%; 21;
68.94% – 70.39%; 24;
70.39% – 73.23%; 28;
73.23% – 74.76%; 32;
74.76% – 75.55%; 35;
75.55% – 79.78%; 39;
79.78% – 80.73%; 43;
80.73% – 82.81%; 46;
82.81% – 85.80%; 50;
85.80% – 87.64%; 54;
87.64% – 91.03%; 57;
91.03% – 92.89%; 61;
92.89% – 96.23%; 65;
96.23% – 99.00%; 68;
99.00% – 100.54%; 72;
100.54% – 102.92%; 76;
> 102.92%; 80;
   Процент выполнения плана (%)  Баллы за выполнение
0                      ≤ 49.29%            10.000000
1               49.29% – 58.12%            12.352941
2               58.12% – 64.45%            14.705882
3               64.45% – 68.94%            17.058824
4               68.94% – 70.39%            19.411765
5               70.39% – 73.23%            21.764706
6               73.23% – 74.76%            24.117647
7               74.76% – 75.55%            26.470588
8               

In [343]:
#DolPPS200
# Отфильтровать DataFrame, оставив только ненулевые значения в столбце sredFact
result = sorted_column_to_string(df, 'DolPPS200Vip')
print(result)
df_nonzero = df[df['DolPPS200Vip'] >= 0.002]
analyze_distribution(df_nonzero, 'DolPPS200Vip')

12.9700272479564; 15.5; 34.7333333333333; 44.0772078265468; 44.813490035769; 50.0; 50.4692703602785; 52.6333333333333; 53.0512820512821; 57.7; 60.86; 61.6249121985484; 61.893955023726; 63.64; 63.64; 64.1; 66.6666666666667; 66.6666666666667; 68.6333333333333; 69.24; 70.1666666666667; 70.6478733926805; 71.3488677387594; 74.0666666666667; 75.0; 75.0; 76.2813401375638; 76.4801035263669; 76.7448767448767; 77.78; 77.8086882088594; 78.26; 81.26; 83.1261950286807; 83.72; 86.9666666666667; 87.0976467406005; 88.5450461692039; 90.0; 91.9982604914112; 95.0187090739009; 100.0; 100.0; 101.0; 103.54110581901; 104.965357967667; 105.26; 109.242871189774; 116.875602700096; 121.74; 126.32; 128.58; 173.34
Минимум: 12.9700272479564
Максимум: 173.34
Медиана (Q2): 76.2813401375638
Q1 (25-й перцентиль): 63.64
Q3 (75-й перцентиль): 91.9982604914112
Межквартильный размах (IQR): 28.3582604914112
Тип скошенности: правостороннее скошенное распределение
Выбросы: 126.32, 109.242871189774, 173.34, 121.74, 128.58, 105

In [344]:
scale_variant3 = build_scale(df_nonzero['DolPPS200Vip'],num_grades=10, min_score=10, max_score=80, scale_type='log')
print(scale_variant3)

  Процент выполнения плана (%)  Баллы за выполнение
0                     ≤ 48.92%                   12
1              48.92% – 59.58%                   12
2              59.58% – 65.11%                   12
3              65.11% – 70.79%                   12
4              70.79% – 76.48%                   12
5              76.48% – 80.65%                   12
6              80.65% – 87.96%                   12
7             87.96% – 100.40%                   12
8            100.40% – 110.73%                   12
9                    > 110.73%                   12


In [345]:
scale_variant3 = build_scale(df_nonzero['DolPPS200Vip'],num_grades=10, min_score=10, max_score=80, scale_type='log')
adjusted_scale_df = adjust_scores(scale_variant3, min_score=10, max_score=80)
print(adjusted_scale_df)
formatted_scale = format_scale(adjusted_scale_df, "scaleDolPPS200Vip")
print(formatted_scale)

  Процент выполнения плана (%)  Баллы за выполнение
0                     ≤ 48.92%            10.000000
1              48.92% – 59.58%            15.714286
2              59.58% – 65.11%            21.428571
3              65.11% – 70.79%            27.142857
4              70.79% – 76.48%            32.857143
5              76.48% – 80.65%            38.571429
6              80.65% – 87.96%            44.285714
7             87.96% – 100.40%            50.000000
8            100.40% – 110.73%            65.000000
9                    > 110.73%            80.000000
scaleDolPPS200Vip = [
    (110.73, 80.000000),  # Если значение больше 110.73%, то 80.0 баллов
    (100.40, 65.000000),  # Если значение в диапазоне 100.40% – 110.73%, то 65.0 баллов
    (87.96, 50.000000),  # Если значение в диапазоне 87.96% – 100.40%, то 50.0 баллов
    (80.65, 44.285714),  # Если значение в диапазоне 80.65% – 87.96%, то 44.285714285714285 баллов
    (76.48, 38.571429),  # Если значение в диапазоне 76.48% 

In [346]:
#DolStorMag
# Отфильтровать DataFrame, оставив только ненулевые значения в столбце sredFact
result = sorted_column_to_string(df, 'DolStorMagVip')
print(result)
df_nonzero = df[df['DolStorMagVip'] >= 0.002]
analyze_distribution(df_nonzero, 'DolStorMagVip')

0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 29.584487534626; 40.3225806451613; 42.857142857143; 43.731778425656; 46.6507177033493; 54.7979797979798; 63.4615384615384; 66.2598081952921; 68.2926829268292; 70.6896551724138; 71.6532905296951; 72.7272727272727; 73.5677083333335; 74.2857142857143; 80.2884615384615; 83.3333333333333; 84.3243243243245; 87.3563218390805; 94.8306595365418; 100.871271585557; 100.969696969697; 101.5625; 102.409638554217; 103.030303030303; 103.448275862069; 104.545454545455; 104.651162790698; 106.611570247934; 107.0; 108.204633204633; 109.523809523809; 110.798122065728; 113.916256157636; 117.818181818182; 119.191919191919; 119.444444444444; 120.476190476191; 137.728937728938; 182.142857142857; 185.714285714286; 196.551724137931; 199.111111111111; 217.948717948718; 228.571428571429; 270.0; 345.098039215686
Минимум: 29.584487534626
Максимум: 345.098039215686
Медиана (Q2): 102.71997079226
Q1 (25-й перцентиль): 72.9373816287879
Q3 (75-й перцентиль): 118.84848484848474
Межкварти

In [347]:
scale_variant3 = build_scale(df_nonzero['DolStorMagVip'],num_grades=20, min_score=10, max_score=90, scale_type='log')
print(scale_variant3)

   Процент выполнения плана (%)  Баллы за выполнение
0                      ≤ 43.07%                   12
1               43.07% – 50.56%                   12
2               50.56% – 65.55%                   12
3               65.55% – 70.69%                   12
4               70.69% – 72.94%                   12
5               72.94% – 77.23%                   12
6               77.23% – 84.08%                   12
7               84.08% – 94.83%                   12
8              94.83% – 101.12%                   12
9             101.12% – 102.72%                   12
10            102.72% – 104.27%                   12
11            104.27% – 106.61%                   12
12            106.61% – 108.53%                   12
13            108.53% – 112.35%                   12
14            112.35% – 118.85%                   12
15            118.85% – 120.48%                   12
16            120.48% – 183.03%                   12
17            183.03% – 197.83%               

In [348]:
scale_variant3 = build_scale(df_nonzero['DolStorMagVip'],num_grades=20, min_score=10, max_score=90, scale_type='log')
adjusted_scale_df = adjust_scores(scale_variant3, min_score=10, max_score=90)
print(adjusted_scale_df)
formatted_scale = format_scale(adjusted_scale_df, "scaleDolStorMagVip")
print(formatted_scale)

   Процент выполнения плана (%)  Баллы за выполнение
0                      ≤ 43.07%            10.000000
1               43.07% – 50.56%            15.000000
2               50.56% – 65.55%            20.000000
3               65.55% – 70.69%            25.000000
4               70.69% – 72.94%            30.000000
5               72.94% – 77.23%            35.000000
6               77.23% – 84.08%            40.000000
7               84.08% – 94.83%            45.000000
8              94.83% – 101.12%            50.000000
9             101.12% – 102.72%            53.636364
10            102.72% – 104.27%            57.272727
11            104.27% – 106.61%            60.909091
12            106.61% – 108.53%            64.545455
13            108.53% – 112.35%            68.181818
14            112.35% – 118.85%            71.818182
15            118.85% – 120.48%            75.454545
16            120.48% – 183.03%            79.090909
17            183.03% – 197.83%            82.

In [354]:
#DolCelevikov
# Отфильтровать DataFrame, оставив только ненулевые значения в столбце sredFact
result = sorted_column_to_string(df, 'DolCelevikovVip')
print(result)
df_nonzero = df[df['DolCelevikovVip'] >= 0.002]
analyze_distribution(df_nonzero, 'DolCelevikovVip')

0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 5.92012797152549; 14.6068492822393; 23.7271301392631; 25.5938014183584; 26.8497553691005; 42.5305800386722; 46.0700649631767; 46.9052378700353; 52.827130689498; 54.0993279936463; 55.1477623930784; 59.1692139660565; 61.6806399592863; 62.5336540452903; 66.6641973479517; 66.8384669818292; 69.3474155287874; 69.3745816277931; 70.6350607089161; 72.5414082078275; 74.8610854571994; 78.7336937611641; 81.5293251114614; 86.1293941801706; 88.6141841149023; 97.0923889339099; 97.5538435514604; 100.806458116598; 105.634597531005; 112.544453120641; 118.251837324166; 119.425891715219; 121.022709194133; 121.212075711939; 130.651437768163; 131.39031595661; 131.677335496499; 134.815072704242; 135.935314273421; 141.859170356974; 143.018622937766; 152.788350634225; 169.637168996534; 175.324598020105; 183.206117465157
Минимум: 5.92012797152549
Максимум: 183.206117465157
Медиана (Q2): 81.5293251114614
Q1 (25-й перцентиль): 59.1692139660565
Q3 (75-й перцентиль): 121.2120

In [358]:
# Пример использования функции с вариантом №1 и 4 секциями (квартилями)
scale_variant1_quartiles = build_scale_quartiles(df_nonzero['DolCelevikovVip'], num_sections=10,min_score=10, max_score=80)

print("Шкала с использованием квартилей (10 секции):")
print("Процент выполнения плана (%); Баллы за выполнение;") 
# Форматирование вывода
output = "\n".join(f"{row['Процент выполнения плана (%)']}; {row['Баллы за выполнение']};" for _, row in scale_variant1_quartiles.iterrows())
print(output)
adjusted_scale_df = adjust_scores(scale_variant1_quartiles, min_score=10, max_score=80)
print(adjusted_scale_df)
formatted_scale = format_scale(adjusted_scale_df, "scaleDolCelevikovVip")
print(formatted_scale)


Шкала с использованием квартилей (10 секции):
Процент выполнения плана (%); Баллы за выполнение;
≤ 33.12%; 10;
33.12% – 53.84%; 17;
53.84% – 63.36%; 25;
63.36% – 70.13%; 33;
70.13% – 81.53%; 41;
81.53% – 98.85%; 48;
98.85% – 119.19%; 56;
119.19% – 131.45%; 64;
131.45% – 142.55%; 72;
> 142.55%; 80;
  Процент выполнения плана (%)  Баллы за выполнение
0                     ≤ 33.12%            10.000000
1              33.12% – 53.84%            16.666667
2              53.84% – 63.36%            23.333333
3              63.36% – 70.13%            30.000000
4              70.13% – 81.53%            36.666667
5              81.53% – 98.85%            43.333333
6             98.85% – 119.19%            50.000000
7            119.19% – 131.45%            60.000000
8            131.45% – 142.55%            70.000000
9                    > 142.55%            80.000000
scaleDolCelevikovVip = [
    (142.55, 80.000000),  # Если значение больше 142.55%, то 80.0 баллов
    (131.45, 70.000000),  # Есл

In [359]:
#Zashchity
# Отфильтровать DataFrame, оставив только ненулевые значения в столбце sredFact
result = sorted_column_to_string(df, 'ZashchityVip')
print(result)
df_nonzero = df[df['ZashchityVip'] >= 0.002]
analyze_distribution(df_nonzero, 'ZashchityVip')

0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 50.0; 66.6666666666667; 66.6666666666667; 80.0; 80.0; 100.0; 100.0; 100.0; 100.0; 133.333333333333; 150.0; 150.0; 200.0; 200.0; 200.0; 200.0; 200.0; 200.0
Минимум: 50.0
Максимум: 200.0
Медиана (Q2): 116.6666666666665
Q1 (25-й перцентиль): 85.0
Q3 (75-й перцентиль): 200.0
Межквартильный размах (IQR): 115.0
Тип скошенности: правостороннее скошенное распределение
Выбросы: 200.0, 200.0, 200.0, 200.0, 200.0, 133.333333333333, 200.0, 150.0, 150.0


In [360]:
kol_inostr_prepod_vip = df_nonzero['ZashchityVip'].tolist()
kol_inostr_prepod_vip_unique = list(set(kol_inostr_prepod_vip))

In [362]:
scale_variant3 = build_scale(df_nonzero['ZashchityVip'],num_grades=10, min_score=10, max_score=80, scale_type='log')
print(scale_variant3)

  Процент выполнения плана (%)  Баллы за выполнение
0                     ≤ 59.43%                   12
1              59.43% – 69.15%                   12
2              69.15% – 77.14%                   12
3              77.14% – 87.47%                   12
4             87.47% – 100.00%                   12
5            100.00% – 118.85%                   12
6            118.85% – 136.51%                   12
7            136.51% – 146.51%                   12
8            146.51% – 168.30%                   12
9                    > 168.30%                   12


In [363]:
scale_variant3 = build_scale(df_nonzero['ZashchityVip'],num_grades=10, min_score=10, max_score=80, scale_type='log')
adjusted_scale_df = adjust_scores(scale_variant3, min_score=10, max_score=80)
print(adjusted_scale_df)
formatted_scale = format_scale(adjusted_scale_df, "scaleZashchityVip")
print(formatted_scale)

  Процент выполнения плана (%)  Баллы за выполнение
0                     ≤ 59.43%                 10.0
1              59.43% – 69.15%                 20.0
2              69.15% – 77.14%                 30.0
3              77.14% – 87.47%                 40.0
4             87.47% – 100.00%                 50.0
5            100.00% – 118.85%                 56.0
6            118.85% – 136.51%                 62.0
7            136.51% – 146.51%                 68.0
8            146.51% – 168.30%                 74.0
9                    > 168.30%                 80.0
scaleZashchityVip = [
    (168.30, 80.000000),  # Если значение больше 168.30%, то 80.0 баллов
    (146.51, 74.000000),  # Если значение в диапазоне 146.51% – 168.30%, то 74.0 баллов
    (136.51, 68.000000),  # Если значение в диапазоне 136.51% – 146.51%, то 68.0 баллов
    (118.85, 62.000000),  # Если значение в диапазоне 118.85% – 136.51%, то 62.0 баллов
    (100.00, 56.000000),  # Если значение в диапазоне 100.00% – 118.8

In [385]:
#DolPPS200
result = sorted_column_to_string(df, 'DolPPS200Vip')
print(result)
df_nonzero = df[df['DolPPS200Vip'] >= 0.002]
analyze_distribution(df_nonzero, 'DolPPS200Vip')

12.9700272479564; 15.5; 34.7333333333333; 44.0772078265468; 44.813490035769; 50.0; 50.4692703602785; 52.6333333333333; 53.0512820512821; 57.7; 60.86; 61.6249121985484; 61.893955023726; 63.64; 63.64; 64.1; 66.6666666666667; 66.6666666666667; 68.6333333333333; 69.24; 70.1666666666667; 70.6478733926805; 71.3488677387594; 74.0666666666667; 75.0; 75.0; 76.2813401375638; 76.4801035263669; 76.7448767448767; 77.78; 77.8086882088594; 78.26; 81.26; 83.1261950286807; 83.72; 86.9666666666667; 87.0976467406005; 88.5450461692039; 90.0; 91.9982604914112; 95.0187090739009; 100.0; 100.0; 101.0; 103.54110581901; 104.965357967667; 105.26; 109.242871189774; 116.875602700096; 121.74; 126.32; 128.58; 173.34
Минимум: 12.9700272479564
Максимум: 173.34
Медиана (Q2): 76.2813401375638
Q1 (25-й перцентиль): 63.64
Q3 (75-й перцентиль): 91.9982604914112
Межквартильный размах (IQR): 28.3582604914112
Тип скошенности: правостороннее скошенное распределение
Выбросы: 126.32, 109.242871189774, 173.34, 121.74, 128.58, 105

In [388]:
# Пример использования функции с вариантом №1 и 4 секциями (квартилями)
scale_variant1_quartiles = build_scale_quartiles(df_nonzero['DolPPS200Vip'], num_sections=10,min_score=30, max_score=80)
print("Шкала с использованием квартилей (10 секции):")
print("Процент выполнения плана (%); Баллы за выполнение;") 
# Форматирование вывода
output = "\n".join(f"{row['Процент выполнения плана (%)']}; {row['Баллы за выполнение']};" for _, row in scale_variant1_quartiles.iterrows())
print(output)
adjusted_scale_df = adjust_scores(scale_variant3, min_score=10, max_score=80)
print(adjusted_scale_df)
formatted_scale = format_scale(adjusted_scale_df, "scaleDolPPS200Vip")
print(formatted_scale)

Шкала с использованием квартилей (10 секции):
Процент выполнения плана (%); Баллы за выполнение;
≤ 48.96%; 30;
48.96% – 59.60%; 35;
59.60% – 65.13%; 41;
65.13% – 70.79%; 46;
70.79% – 76.48%; 52;
76.48% – 80.66%; 57;
80.66% – 87.97%; 63;
87.97% – 100.40%; 68;
100.40% – 110.77%; 74;
> 110.77%; 80;
   Процент выполнения плана (%)  Баллы за выполнение
0                      ≤ 32.54%            10.000000
1               32.54% – 50.58%            13.636364
2               50.58% – 57.51%            17.272727
3               57.51% – 64.69%            20.909091
4               64.69% – 66.61%            24.545455
5               66.61% – 70.72%            28.181818
6               70.72% – 74.56%            31.818182
7               74.56% – 77.59%            35.454545
8               77.59% – 87.78%            39.090909
9               87.78% – 91.20%            42.727273
10              91.20% – 97.80%            46.363636
11             97.80% – 107.26%            50.000000
12            

In [366]:
#ZayavAktiv
result = sorted_column_to_string(df, 'ZayavAktivVip')
print(result)
df_nonzero = df[df['ZayavAktivVip'] >= 0.002]
analyze_distribution(df_nonzero, 'ZayavAktivVip')

0.0; 0.0; 0.0; 0.0; 16.6666666666667; 22.2222222222222; 25.0; 33.3333333333333; 33.3333333333333; 40.0; 50.0; 50.0; 50.0; 50.0; 50.0; 66.6666666666667; 66.6666666666667; 66.6666666666667; 71.4285714285714; 75.0; 75.0; 75.0; 75.0; 75.0; 75.0; 80.0; 83.3333333333333; 87.5; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 112.5; 116.666666666667; 118.181818181818; 120.0; 120.0; 125.0; 125.0; 133.333333333333; 133.333333333333; 140.0; 162.5; 200.0; 200.2002002002; 210.0; 216.666666666667; 233.333333333333; 260.0; 450.0; 1601.6016016016
Минимум: 16.6666666666667
Максимум: 1601.6016016016
Медиана (Q2): 100.0
Q1 (25-й перцентиль): 66.6666666666667
Q3 (75-й перцентиль): 125.0
Межквартильный размах (IQR): 58.3333333333333
Тип скошенности: правостороннее скошенное распределение
Выбросы: 200.0, 450.0, 133.333333333333, 118.181818181818, 133.333333333333, 112.5, 260.0, 120.0, 1601.6016016016, 216.666666666667, 200.2002002002, 120.0, 125.0, 162.5, 233.333333333333, 210.0, 116.666666666667, 140.0, 125.0


In [367]:
# Пример использования функции с вариантом №1 и 4 секциями (квартилями)
scale_variant1_quartiles = build_scale_quartiles(df_nonzero['ZayavAktivVip'], num_sections=20,min_score=10, max_score=90)
print("Шкала с использованием квартилей (10 секции):")
print("Процент выполнения плана (%); Баллы за выполнение;") 
# Форматирование вывода
output = "\n".join(f"{row['Процент выполнения плана (%)']}; {row['Баллы за выполнение']};" for _, row in scale_variant1_quartiles.iterrows())
print(output)

Шкала с использованием квартилей (10 секции):
Процент выполнения плана (%); Баллы за выполнение;
≤ 23.33%; 10;
23.33% – 31.67%; 14;
31.67% – 42.00%; 18;
42.00% – 60.00%; 22;
60.00% – 71.43%; 26;
71.43% – 77.00%; 31;
77.00% – 82.67%; 35;
82.67% – 90.00%; 39;
90.00% – 107.50%; 43;
107.50% – 116.67%; 47;
116.67% – 118.91%; 52;
118.91% – 124.00%; 56;
124.00% – 134.67%; 60;
134.67% – 153.50%; 64;
153.50% – 200.00%; 68;
200.00% – 204.12%; 73;
204.12% – 215.33%; 77;
215.33% – 238.67%; 81;
238.67% – 374.00%; 85;
> 374.00%; 90;


In [368]:
adjusted_scale_df = adjust_scores(scale_variant1_quartiles, min_score=10, max_score=90)
print(adjusted_scale_df)
formatted_scale = format_scale(adjusted_scale_df, "scaleZayavAktivVip")
print(formatted_scale)

   Процент выполнения плана (%)  Баллы за выполнение
0                      ≤ 23.33%            10.000000
1               23.33% – 31.67%            15.000000
2               31.67% – 42.00%            20.000000
3               42.00% – 60.00%            25.000000
4               60.00% – 71.43%            30.000000
5               71.43% – 77.00%            35.000000
6               77.00% – 82.67%            40.000000
7               82.67% – 90.00%            45.000000
8              90.00% – 107.50%            50.000000
9             107.50% – 116.67%            53.636364
10            116.67% – 118.91%            57.272727
11            118.91% – 124.00%            60.909091
12            124.00% – 134.67%            64.545455
13            134.67% – 153.50%            68.181818
14            153.50% – 200.00%            71.818182
15            200.00% – 204.12%            75.454545
16            204.12% – 215.33%            79.090909
17            215.33% – 238.67%            82.

In [369]:
#Innovacii
result = sorted_column_to_string(df, 'InnovaciiVip')
print(result)
df_nonzero = df[df['InnovaciiVip'] >= 0.002]
analyze_distribution(df_nonzero, 'InnovaciiVip')

0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 12.5; 25.0; 25.0; 30.0; 33.3333333333333; 33.3333333333333; 33.3333333333333; 40.0; 50.0; 50.0; 50.0; 60.0; 66.6666666666667; 66.6666666666667; 75.0; 80.0; 80.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 125.0; 133.333333333333; 133.333333333333; 133.333333333333; 133.333333333333; 150.0; 150.0; 160.0; 166.666666666667; 200.0; 200.0; 242.857142857143; 325.0; 500.0; 550.0
Минимум: 12.5
Максимум: 550.0
Медиана (Q2): 100.0
Q1 (25-й перцентиль): 52.5
Q3 (75-й перцентиль): 133.333333333333
Межквартильный размах (IQR): 80.833333333333
Тип скошенности: правостороннее скошенное распределение
Выбросы: 133.333333333333, 150.0, 200.0, 160.0, 500.0, 133.333333333333, 550.0, 200.0, 242.857142857143, 133.333333333333, 133.333333333333, 166.666666666667, 325.0, 150.0, 125.0


In [370]:
scale_variant3 = build_scale(df_nonzero['InnovaciiVip'],num_grades=20, min_score=10, max_score=90, scale_type='log')
print(scale_variant3)

   Процент выполнения плана (%)  Баллы за выполнение
0                      ≤ 25.00%                   11
1               25.00% – 30.00%                   11
2               30.00% – 33.33%                   11
3               33.33% – 40.00%                   11
4               40.00% – 50.00%                   12
5               50.00% – 60.00%                   12
6               60.00% – 66.67%                   12
7               66.67% – 75.00%                   12
8               75.00% – 80.00%                   12
9              80.00% – 100.00%                   12
10            100.00% – 125.00%                   12
11            125.00% – 133.33%                   12
12            133.33% – 150.00%                   12
13            150.00% – 160.00%                   12
14            160.00% – 166.67%                   12
15            166.67% – 200.00%                   12
16            200.00% – 242.86%                   12
17            242.86% – 325.00%               

In [371]:
scale_variant3 = build_scale(df_nonzero['InnovaciiVip'],num_grades=20, min_score=10, max_score=90, scale_type='log')
adjusted_scale_df = adjust_scores(scale_variant3, min_score=10, max_score=80)
print(adjusted_scale_df)
formatted_scale = format_scale(adjusted_scale_df, "scaleInnovaciiVip")
print(formatted_scale)

   Процент выполнения плана (%)  Баллы за выполнение
0                      ≤ 25.00%            10.000000
1               25.00% – 30.00%            14.444444
2               30.00% – 33.33%            18.888889
3               33.33% – 40.00%            23.333333
4               40.00% – 50.00%            27.777778
5               50.00% – 60.00%            32.222222
6               60.00% – 66.67%            36.666667
7               66.67% – 75.00%            41.111111
8               75.00% – 80.00%            45.555556
9              80.00% – 100.00%            50.000000
10            100.00% – 125.00%            53.000000
11            125.00% – 133.33%            56.000000
12            133.33% – 150.00%            59.000000
13            150.00% – 160.00%            62.000000
14            160.00% – 166.67%            65.000000
15            166.67% – 200.00%            68.000000
16            200.00% – 242.86%            71.000000
17            242.86% – 325.00%            74.

In [372]:
#KolInostrPrepod
result = sorted_column_to_string(df, 'KolInostrPrepodVip')
print(result)
df_nonzero = df[df['KolInostrPrepodVip'] >= 0.002]

analyze_distribution(df_nonzero, 'KolInostrPrepodVip')

0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 100.0; 200.0; 200.0; 200.0; 200.0; 200.0; 200.0; 200.0; 200.0; 200.0; 200.0; 200.0; 300.0; 300.0; 300.0; 300.0; 300.0; 400.0; 400.0; 500.0; 800.0
Минимум: 100.0
Максимум: 800.0
Медиана (Q2): 100.0
Q1 (25-й перцентиль): 100.0
Q3 (75-й перцентиль): 200.0
Межквартильный размах (IQR): 100.0
Тип скошенности: правостороннее скошенное распределение
Выбросы: 200.0, 200.0, 400.0, 200.0, 300.0, 400.0, 200.0, 200.0, 500.0, 200.0, 300.0, 200.0, 200.0, 200.0, 300.0, 300.0, 300.0, 200.0, 200.0, 800.0


In [180]:
kol_inostr_prepod_vip = df_nonzero['KolInostrPrepodVip'].tolist()
kol_inostr_prepod_vip_unique = list(set(kol_inostr_prepod_vip))

In [181]:
print(kol_inostr_prepod_vip_unique)

[800.0, 100.0, 200.0, 300.0, 400.0, 500.0]


In [373]:
scale_variant3 = build_scale(df_nonzero['KolInostrPrepodVip'],num_grades=20, min_score=10, max_score=90, scale_type='log')
print(scale_variant3)

   Процент выполнения плана (%)  Баллы за выполнение
0                     ≤ 118.96%                   12
1             118.96% – 141.48%                   12
2             141.48% – 168.23%                   12
3             168.23% – 200.00%                   12
4             200.00% – 221.35%                   12
5             221.35% – 244.97%                   12
6             244.97% – 271.10%                   12
7             271.10% – 300.00%                   12
8             300.00% – 322.38%                   12
9             322.38% – 346.42%                   12
10            346.42% – 372.25%                   12
11            372.25% – 400.00%                   12
12            400.00% – 422.95%                   12
13            422.95% – 447.22%                   12
14            447.22% – 472.88%                   12
15            472.88% – 500.00%                   12
16            500.00% – 562.36%                   13
17            562.36% – 632.48%               

In [374]:
scale_variant3 = build_scale(df_nonzero['KolInostrPrepodVip'],num_grades=20, min_score=10, max_score=90, scale_type='log')
adjusted_scale_df = adjust_scores(scale_variant3, min_score=10, max_score=90)
print(adjusted_scale_df)
formatted_scale = format_scale(adjusted_scale_df, "scaleKolInostrPrepodVip")
print(formatted_scale)

   Процент выполнения плана (%)  Баллы за выполнение
0                     ≤ 118.96%            50.000000
1             118.96% – 141.48%            52.105263
2             141.48% – 168.23%            54.210526
3             168.23% – 200.00%            56.315789
4             200.00% – 221.35%            58.421053
5             221.35% – 244.97%            60.526316
6             244.97% – 271.10%            62.631579
7             271.10% – 300.00%            64.736842
8             300.00% – 322.38%            66.842105
9             322.38% – 346.42%            68.947368
10            346.42% – 372.25%            71.052632
11            372.25% – 400.00%            73.157895
12            400.00% – 422.95%            75.263158
13            422.95% – 447.22%            77.368421
14            447.22% – 472.88%            79.473684
15            472.88% – 500.00%            81.578947
16            500.00% – 562.36%            83.684211
17            562.36% – 632.48%            85.

In [375]:
#KolPublik
result = sorted_column_to_string(df, 'KolPublikVip')
print(result)
df_nonzero = df[df['KolPublikVip'] >= 0.002]
analyze_distribution(df_nonzero, 'KolPublikVip')

30.382705229331; 33.7662337662338; 40.4595404595404; 48.3389122844905; 50.1257022253951; 55.6242847864411; 57.2080123266562; 57.7533526310637; 59.4017094017092; 59.9442379182156; 61.0795454545455; 61.4547357404502; 63.8297872340425; 64.2188196818789; 66.3937988988696; 67.5513921415563; 68.2044323835368; 69.2969930258067; 70.7223060164237; 73.6310653293577; 76.3760049474338; 76.5179016300094; 77.1031746031744; 78.7878787878788; 79.3036750483558; 81.7957090168814; 81.9386331938635; 82.5154489276624; 83.0744208353317; 86.5066905883232; 89.0581866191622; 91.0220565392978; 94.6716834567296; 94.8783610755442; 95.7072484166082; 100.635407280855; 104.062643119646; 109.375; 109.772727272728; 113.038950804908; 114.390659845205; 115.528230219533; 122.49299650445; 122.957942301008; 127.096151526119; 130.039894745777; 143.035313401989; 143.790849673203; 150.964066608238; 166.064981949458; 218.636363636364; 344.764369675402; 398.034398034398
Минимум: 30.382705229331
Максимум: 398.034398034398
Медиан

In [376]:
scale_variant3 = build_scale(df_nonzero['KolPublikVip'],num_grades=20, min_score=10, max_score=90, scale_type='log')
print(scale_variant3)

   Процент выполнения плана (%)  Баллы за выполнение
0                      ≤ 45.02%                   12
1               45.02% – 55.94%                   12
2               55.94% – 59.07%                   12
3               59.07% – 61.23%                   12
4               61.23% – 64.22%                   12
5               64.22% – 67.94%                   12
6               67.94% – 71.29%                   12
7               71.29% – 76.49%                   12
8               76.49% – 78.99%                   12
9               78.99% – 81.94%                   12
10              81.94% – 85.12%                   12
11              85.12% – 91.74%                   12
12              91.74% – 95.54%                   12
13             95.54% – 106.16%                   12
14            106.16% – 113.04%                   12
15            113.04% – 119.66%                   12
16            119.66% – 127.68%                   12
17            127.68% – 143.64%               

In [377]:
scale_variant3 = build_scale(df_nonzero['KolPublikVip'],num_grades=20, min_score=10, max_score=90, scale_type='log')
adjusted_scale_df = adjust_scores(scale_variant3, min_score=10, max_score=80)
print(adjusted_scale_df)
formatted_scale = format_scale(adjusted_scale_df, "scaleKolPublikVip")
print(formatted_scale)

   Процент выполнения плана (%)  Баллы за выполнение
0                      ≤ 45.02%            10.000000
1               45.02% – 55.94%            13.076923
2               55.94% – 59.07%            16.153846
3               59.07% – 61.23%            19.230769
4               61.23% – 64.22%            22.307692
5               64.22% – 67.94%            25.384615
6               67.94% – 71.29%            28.461538
7               71.29% – 76.49%            31.538462
8               76.49% – 78.99%            34.615385
9               78.99% – 81.94%            37.692308
10              81.94% – 85.12%            40.769231
11              85.12% – 91.74%            43.846154
12              91.74% – 95.54%            46.923077
13             95.54% – 106.16%            50.000000
14            106.16% – 113.04%            55.000000
15            113.04% – 119.66%            60.000000
16            119.66% – 127.68%            65.000000
17            127.68% – 143.64%            70.

In [378]:
#ObemNIOKR
result = sorted_column_to_string(df, 'ObemNIOKRVip')
print(result)
df_nonzero = df[df['ObemNIOKRVip'] >= 0.002]
analyze_distribution(df_nonzero, 'ObemNIOKRVip')

0.0004121212121212; 0.0014285714285714; 0.0014285714285714; 0.0014285714285714; 0.0014285714285714; 0.0014285714285714; 0.0014285714285714; 3.24306794227339; 5.38742345737163; 8.05933437458522; 8.82813480913867; 9.67866821525358; 18.1693638392857; 19.2687994054704; 22.2541718335393; 28.6681335676629; 34.0002492480192; 36.3662250159083; 48.8482435966402; 51.624388518024; 52.7530664729888; 66.5390569655308; 68.5652203178412; 72.4517466267671; 73.6363636363635; 74.0462530241167; 80.1316130260653; 80.5005537826369; 81.0153929246557; 83.4331354360532; 83.7788124408261; 87.9284649776452; 99.939555383857; 106.63416490106; 112.83329847342; 113.393240725953; 117.21396858787; 117.71995007966; 127.87969028176; 134.778837441114; 169.101508126863; 172.439860074436; 179.793402672308; 181.083292826689; 205.473252270464; 205.883988527497; 217.917675544794; 276.786145054945; 286.758350916505; 355.679908834023; 565.115319974143; 1292.48130699088; 4323.07248675047
Минимум: 3.24306794227339
Максимум: 4323

In [379]:
scale_variant3 = build_scale(df_nonzero['ObemNIOKRVip'],num_grades=20, min_score=10, max_score=90, scale_type='log')
print(scale_variant3)

   Процент выполнения плана (%)  Баллы за выполнение
0                       ≤ 8.25%                   11
1                8.25% – 13.31%                   11
2               13.31% – 21.47%                   11
3               21.47% – 34.00%                   11
4               34.00% – 49.53%                   12
5               49.53% – 59.25%                   12
6               59.25% – 71.46%                   12
7               71.46% – 74.05%                   12
8               74.05% – 80.63%                   12
9               80.63% – 83.61%                   12
10              83.61% – 96.79%                   12
11             96.79% – 112.83%                   12
12            112.83% – 117.34%                   12
13            117.34% – 131.28%                   12
14            131.28% – 171.60%                   12
15            171.60% – 181.08%                   12
16            181.08% – 208.83%                   12
17            208.83% – 281.73%               

In [380]:
scale_variant3 = build_scale(df_nonzero['ObemNIOKRVip'],num_grades=20, min_score=10, max_score=90, scale_type='log')
adjusted_scale_df = adjust_scores(scale_variant3, min_score=10, max_score=90)
print(adjusted_scale_df)
formatted_scale = format_scale(adjusted_scale_df, "scaleObemNIOKRVip")
print(formatted_scale)

   Процент выполнения плана (%)  Баллы за выполнение
0                       ≤ 8.25%            10.000000
1                8.25% – 13.31%            13.636364
2               13.31% – 21.47%            17.272727
3               21.47% – 34.00%            20.909091
4               34.00% – 49.53%            24.545455
5               49.53% – 59.25%            28.181818
6               59.25% – 71.46%            31.818182
7               71.46% – 74.05%            35.454545
8               74.05% – 80.63%            39.090909
9               80.63% – 83.61%            42.727273
10              83.61% – 96.79%            46.363636
11             96.79% – 112.83%            50.000000
12            112.83% – 117.34%            55.000000
13            117.34% – 131.28%            60.000000
14            131.28% – 171.60%            65.000000
15            171.60% – 181.08%            70.000000
16            181.08% – 208.83%            75.000000
17            208.83% – 281.73%            80.

In [383]:
#ObemPOU
result = sorted_column_to_string(df, 'ObemPOUVip')
print(result)
df_nonzero = df[df['ObemPOUVip'] >= 0.002]
analyze_distribution(df_nonzero, 'ObemPOUVip')

11.1877566799765; 24.8760671550672; 26.3341398176292; 36.6843446873411; 36.9530764976958; 50.489405054085; 50.9480563879993; 55.7759871607739; 57.9445301373628; 61.2282285065716; 63.994521234625; 65.7226271321397; 66.1667291914745; 66.6050221088435; 68.004454057424; 69.882415081827; 71.2755360467173; 72.8950917751169; 74.1360021048961; 76.2768259253037; 76.3820986502424; 77.8882892199825; 80.7111478674245; 86.346040127052; 89.9225450795747; 89.9648026580559; 91.2040013240445; 91.5404006685597; 95.1560613756614; 99.5601447621963; 105.709752027955; 107.067653507046; 108.009237937947; 108.544321650159; 111.994232991409; 116.013330612245; 116.997197110741; 117.000314790841; 118.828509022834; 120.74698518118; 120.978476437158; 123.530786066175; 124.53333632287; 124.978348276757; 135.280389165123; 138.083693589971; 140.596538050734; 142.80646212765; 167.831982625483; 168.567780798255; 172.388463316211; 188.041834901522; 633.447707142857
Минимум: 11.1877566799765
Максимум: 633.447707142857
Ме

In [382]:
scale_variant3 = build_scale(df_nonzero['ObemPOUVip'],num_grades=20, min_score=10, max_score=90, scale_type='log')
print(scale_variant3)

   Процент выполнения плана (%)  Баллы за выполнение
0                      ≤ 32.14%                   11
1               32.14% – 50.58%                   12
2               50.58% – 57.50%                   12
3               57.50% – 64.68%                   12
4               64.68% – 66.61%                   12
5               66.61% – 70.72%                   12
6               70.72% – 74.56%                   12
7               74.56% – 77.58%                   12
8               77.58% – 87.76%                   12
9               87.76% – 91.20%                   12
10              91.20% – 97.77%                   12
11             97.77% – 107.26%                   12
12            107.26% – 111.30%                   12
13            111.30% – 117.00%                   12
14            117.00% – 120.75%                   12
15            120.75% – 124.13%                   12
16            124.13% – 135.84%                   12
17            135.84% – 142.36%               

In [384]:
scale_variant3 = build_scale(df_nonzero['ObemPOUVip'],num_grades=20, min_score=10, max_score=90, scale_type='linear')
adjusted_scale_df = adjust_scores(scale_variant3, min_score=10, max_score=90)
print(adjusted_scale_df)
formatted_scale = format_scale(adjusted_scale_df, "scaleObemPOUVip")
print(formatted_scale)

   Процент выполнения плана (%)  Баллы за выполнение
0                      ≤ 32.54%            10.000000
1               32.54% – 50.58%            13.636364
2               50.58% – 57.51%            17.272727
3               57.51% – 64.69%            20.909091
4               64.69% – 66.61%            24.545455
5               66.61% – 70.72%            28.181818
6               70.72% – 74.56%            31.818182
7               74.56% – 77.59%            35.454545
8               77.59% – 87.78%            39.090909
9               87.78% – 91.20%            42.727273
10              91.20% – 97.80%            46.363636
11             97.80% – 107.26%            50.000000
12            107.26% – 111.30%            55.000000
13            111.30% – 117.00%            60.000000
14            117.00% – 120.75%            65.000000
15            120.75% – 124.13%            70.000000
16            124.13% – 135.84%            75.000000
17            135.84% – 142.36%            80.

In [132]:
#PodgotovkaEncikl
result = sorted_column_to_string(df, 'DolMolNPRVip')
print(result)
df_nonzero = df[df['DolMolNPRVip'] >= 0.002]
analyze_distribution(df_nonzero, 'DolMolNPRVip')

0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0; 0
Минимум: nan
Максимум: nan
Медиана (Q2): nan
Q1 (25-й перцентиль): nan
Q3 (75-й перцентиль): nan
Межквартильный размах (IQR): nan
Тип скошенности: приблизительно симметричное
Выбросы: Нет выбросов


In [133]:
scale_variant3 = build_scale(df_nonzero['DolMolNPRVip'],num_grades=10, min_score=10, max_score=80, scale_type='log')
print(scale_variant3)

ValueError: max() arg is an empty sequence

In [135]:
df_nonzero = df[df['DolMolNPRVip'] >= 0.002]
analyze_distribution(df_nonzero, 'DolMolNPRVip')

Минимум: nan
Максимум: nan
Медиана (Q2): nan
Q1 (25-й перцентиль): nan
Q3 (75-й перцентиль): nan
Межквартильный размах (IQR): nan
Тип скошенности: приблизительно симметричное
Выбросы: Нет выбросов


In [134]:
scale_variant3 = build_scale(df_nonzero['DolMolNPRVip'],num_grades=10, min_score=10, max_score=80, scale_type='log')
adjusted_scale_df = adjust_scores(scale_variant3, min_score=10, max_score=80)
print(adjusted_scale_df)
formatted_scale = format_scale(adjusted_scale_df, "scaleDolMolNPRVip")
print(formatted_scale)

ValueError: max() arg is an empty sequence